In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import requests
#pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 120)
import time
# import haversine as hs
# import geopandas as gpd
# import pyproj
import os
import math
pd.set_option('display.float_format', lambda x: '%.f' % x)
import re

# 1. Preparation

In [830]:
#entries = os.listdir('D:/Transjakarta/Pelanggan/Tap In Out Maret IT')

# JUNI
# dfg = pd.read_csv('GATE 19 - 25 Juni 2023.csv', dtype={'Card Number Var': str})
# dftob = pd.read_csv('TOB 19 - 25 Juni 2023.csv', dtype={'Card Number Var': str})

# APRIL
# dfti = pd.read_csv('../../../Pelanggan/IT April/Tap In dan Out_BRT TOB_3-16 April 2023/20230428 Data Tap IN TOB 3 - 9 April 2023.csv')
# dfto = pd.read_csv('../../../Pelanggan/IT April/Tap In dan Out_BRT TOB_3-16 April 2023/20230428 Data Tap OUT TOB 3 - 9 April 2023.csv')

In [2]:
kategori = pd.read_excel('../Daftar Rute 2707.xlsx')
kategori['Kode Rute'] = kategori['Kode Rute'].apply(lambda x: str(x)) 
kategori.head(1)

,Jenis Layanan,Kategori Rute,Kode Rute
0,BRT,BRT,1


In [3]:
# JULI
gate_in = pd.read_csv('../Juli IT/GATE IN 17 - 24 Juli 2023.csv', dtype={'card_number_var': str})
gate_out = pd.read_csv('../Juli IT/GATE OUT 17 - 24 Juli 2023.csv', dtype={'card_number_var': str})

tob_in = pd.read_csv('../Juli IT/TOB IN 17 - 24 Juli 2023.csv', dtype={'card_number_var': str})
tob_out = pd.read_csv('../Juli IT/TOB OUT 17 - 24 Juli 2023.csv', dtype={'card_number_var': str})


# dftob = pd.read_csv('../TOB 19 - 25 Juni 2023.csv', dtype={'Card Number Var': str})




C:\Users\Nadya\AppData\Local\Temp\ipykernel_21168\331341805.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  tob_in = pd.read_csv('../Juli IT/TOB IN 17 - 24 Juli 2023.csv', dtype={'card_number_var': str})


In [4]:
tob_in.drop('transcode_txt', axis=1, inplace=True)
tob_out.drop('transcode_txt', axis=1, inplace=True)
gate_in.drop('transcode_txt', axis=1, inplace=True)
gate_out.drop('transcode_txt', axis=1, inplace=True)

In [5]:
print (tob_in.columns)
print (tob_out.columns)

Index(['shelter_name_var', 'shelter_code', 'terminal_code', 'waktu_tap',
       'card_number_var', 'tap', 'p_latitude_flo', 'p_longitude_flo'],
      dtype='object')
Index(['shelter_name_var', 'shelter_code', 'terminal_code', 'waktu_tap',
       'card_number_var', 'tap', 'p_latitude_flo', 'p_longitude_flo'],
      dtype='object')


In [6]:
tob_in['Waktu Tap'] = tob_in['waktu_tap'].apply(lambda x: datetime.strptime(x,'%Y/%m/%d %H:%M:%S')) 
tob_out['Waktu Tap'] = tob_out['waktu_tap'].apply(lambda x: datetime.strptime(x,'%Y/%m/%d %H:%M:%S')) 
gate_in['Waktu Tap'] = gate_in['waktu_tap'].apply(lambda x: datetime.strptime(x,'%Y/%m/%d %H:%M:%S')) 
gate_out['Waktu Tap'] = gate_out['waktu_tap'].apply(lambda x: datetime.strptime(x,'%Y/%m/%d %H:%M:%S')) 


tob_in['Hour'] = tob_in['Waktu Tap'].apply(lambda x: x.hour) 
tob_out['Hour'] = tob_out['Waktu Tap'].apply(lambda x: x.hour) 
gate_in['Hour'] = gate_in['Waktu Tap'].apply(lambda x: x.hour) 
gate_out['Hour'] = gate_out['Waktu Tap'].apply(lambda x: x.hour) 

tob_in['Day'] = tob_in['Waktu Tap'].apply(lambda x: x.day) 
tob_out['Day'] = tob_out['Waktu Tap'].apply(lambda x: x.day) 
gate_in['Day'] = gate_in['Waktu Tap'].apply(lambda x: x.day) 
gate_out['Day'] = gate_out['Waktu Tap'].apply(lambda x: x.day) 


In [7]:
gate_out.loc[gate_out['shelter_name_var'].isna(), 'shelter_name_var'] = gate_out[gate_out['terminal_code']==10743]['shelter_name_var'].unique()[0]

In [8]:
tob_in['Type'] = 'TOB'
tob_out['Type'] = 'TOB'
gate_in['Type'] = 'GATE'
gate_out['Type'] = 'GATE'

In [9]:
tob_in.head(2)

,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,p_latitude_flo,p_longitude_flo,Waktu Tap,Hour,Day,Type
0,H195,H195,47659,2023/7/17 00:00:06,9360885010145453,IN,-6.21361,107,2023-07-17 00:00:06,0,17,TOB
1,H204,H204,47668,2023/7/17 00:03:22,6032986090875120,IN,-6.25048,107,2023-07-17 00:03:22,0,17,TOB


In [10]:
gate_in.head(2)

,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,Waktu Tap,Hour,Day,Type
0,ASEAN,1024,10080,2023/7/17 00:00:06,7546140002354405,IN,2023-07-17 00:00:06,0,17,GATE
1,PRJ,12224,10891,2023/7/17 00:00:08,7546270000983465,IN,2023-07-17 00:00:08,0,17,GATE


In [11]:
tob = pd.concat ([tob_in, tob_out])

In [12]:
tob[(tob['shelter_name_var'].isna()) & (tob['shelter_code'].str.contains('B'))]['shelter_code'].unique()

array(['B7353TGD', 'B7802SGX'], dtype=object)

In [13]:
tob_out[(tob_out['shelter_name_var'].isna()) & (tob_out['shelter_code'].str.contains('B'))]['shelter_code'].unique()

array(['B7353TGD', 'B7802SGX'], dtype=object)

In [14]:
gate_in['Waktu Tap'].min(), gate_in['Waktu Tap'].max()

(Timestamp('2023-07-17 00:00:06'), Timestamp('2023-07-24 23:59:57'))

### A. Menyatukan TOB Dengan Data Transit dan Lokasi TOB Portable

#### a. Transit

In [15]:
dft1 = pd.read_excel('../../Python/Data Transit Raw/Juli/Administrator  Transjakarta Transit 17 Juli.xlsx', header=1)
dft2 = pd.read_excel('../../Python/Data Transit Raw/Juli/Administrator  Transjakarta Transit 18 Juli.xlsx', header=1)
dft3 = pd.read_excel('../../Python/Data Transit Raw/Juli/Administrator  Transjakarta Transit 19 Juli.xlsx', header=1)
dft4 = pd.read_excel('../../Python/Data Transit Raw/Juli/Administrator  Transjakarta Transit 20 Juli.xlsx', header=1)
dft5 = pd.read_excel('../../Python/Data Transit Raw/Juli/Administrator  Transjakarta Transit 21 Juli.xlsx', header=1)
dft6 = pd.read_excel('../../Python/Data Transit Raw/Juli/Administrator  Transjakarta Transit 22 Juli.xlsx', header=1)
dft7 = pd.read_excel('../../Python/Data Transit Raw/Juli/Administrator  Transjakarta Transit 23 Juli.xlsx', header=1)
dft8 = pd.read_excel('../../Python/Data Transit Raw/Juli/Administrator  Transjakarta Transit 24 Juli.xlsx', header=1)

In [16]:
dft = pd.concat([dft1, dft2, dft3, dft4, dft5, dft6, dft7, dft8])

In [17]:
dft['Kode Rute'] = dft['Rute Real'].apply(lambda x: x.split('(')[-1].replace(')', ''))
dfta = dft.merge(kategori, on='Kode Rute', how='left')
dfta[dfta['Jenis Layanan'].isna()]['Kode Rute'].unique()

array([], dtype=object)

In [18]:
dfta = dfta[dfta['Jenis Layanan']=='Non-BRT']

In [19]:
dfta.head(1)

,Operator,No Bus,Nik,Pramudi,SPBU,Rute Real,Rute Renops,Rute Status,KM Start,KM End,KM Baku,Kategori DO,No Do,Tanggal Do,Jam Start Do,Jam End Do,Respons Verifikator,Respons Admin,Kode Rute,Jenis Layanan,Kategori Rute
0,BayuHolongPersada SB 31,BHL 220514,2205056,NANANG,NaN,TMII - TEGAL PARANG - TMII VIA CAWANG SUTOYO (7D),TMII - TEGAL PARANG - TMII VIA CAWANG SUTOYO (7D),IncompleteLog0 / 44 Halte,92313,92346,NaN,DO Reguler,202307172055391046,2023-07-17,20:56,22:16,Proses,Proses,7D,Non-BRT,Angkutan Umum Integrasi


In [20]:
dfta['TimeStart'] = dfta['Tanggal Do'] +' '+ dfta['Jam Start Do']  
dfta['TimeEnd'] = dfta['Tanggal Do'] +' '+ dfta['Jam End Do']  
dfta['TimeStart'] = dfta['TimeStart'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M')) 
dfta['Day'] = dfta['TimeStart'].apply(lambda x: x.day)
# dfta[dfta['TimeEnd'].isna()]
# dfta['TimeEnd'] = dfta['TimeEnd'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M')) 

In [21]:
dftb = dfta.groupby(['Kode Rute', 'Day', 'No Bus']).count()[['No Do']].reset_index().drop('No Do', axis=1)
dftb['No Body 2'] = dftb['No Bus'].apply (lambda x: x.replace(' ', ''))
dftb['No Body 2'] = dftb['No Body 2'].apply(lambda x: [re.findall(r'(\w+?)(\d+)', x.replace(' ', ''))][0][0][0] +' '+ [re.findall(r'(\w+?)(\d+)', x.replace(' ', ''))][0][0][1])
dftb['No Body 2'] = dftb['No Body 2'].apply(lambda x: x.upper().strip().replace(' 0', ' '))

In [22]:
# dftb

In [23]:
tob['shelter_name_var'] = tob['shelter_name_var'].fillna(tob['shelter_code']) 

tob['No Body 2'] = tob['shelter_name_var'].apply(lambda x: x.replace(' ', ''))
tob['No Body 2'] = tob['shelter_name_var'].apply(lambda x: [re.findall(r'(\w+?)(\d+)', x.replace(' ', ''))][0][0][0] +' '+ [re.findall(r'(\w+?)(\d+)', x.replace(' ', ''))][0][0][1])
tob['No Body 2'] = tob['No Body 2'].apply(lambda x: x.upper().strip().replace(' 0', ' '))
# tob['No Body 2']

In [24]:
tob_2 = tob.merge(dftb, on=['No Body 2', 'Day'], how='left')

In [25]:
tob_3 = tob_2[(tob_2['Kode Rute'].isna())]

In [26]:
tob_3.drop(['Kode Rute', 'No Bus'], inplace=True, axis=1)

C:\Users\Nadya\AppData\Local\Temp\ipykernel_21168\2050432657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tob_3.drop(['Kode Rute', 'No Bus'], inplace=True, axis=1)


In [255]:
# tob_4[(tob_4['No Body 2'].str.contains('H')==False) & tob_4['Kode Rute_y'].isna()]

#### b. Open and Merge from Realisasi Bus

In [27]:
realisasibus = pd.read_csv('../Juli IT/Realisasi Bus 17-24 Juli.csv')

realisasibus['Tanggal'] = realisasibus['Tanggal'].apply(lambda x: datetime.strptime(x, '%b %d, %Y'))
realisasibus['No Body 2'] = realisasibus['No Body'].apply(lambda x: x.upper().replace(' ', ''))
realisasibus['No Body 2'] = realisasibus['No Body 2'].apply(lambda x: [re.findall(r'(\w+?)(\d+)', x.replace(' ', ''))][0][0][0] +' '+ [re.findall(r'(\w+?)(\d+)', x.replace(' ', ''))][0][0][1])
realisasibus['No Body 2'] = realisasibus['No Body 2'].apply(lambda x: x.upper().strip().replace(' 0', ' '))

In [28]:
realisasibus = realisasibus[['Tanggal', 'Kode Rute', 'No Body', 'No Body 2']]
realisasibus['Day'] = realisasibus['Tanggal'].apply(lambda x: x.day)

In [29]:
tob_4 = tob_3.merge(realisasibus, on=['No Body 2', 'Day'], how='left')

tob_4[(tob_4['Kode Rute'].isna()) & (tob_4['No Body 2'].str.contains('H')==False)]['No Body 2'].unique()

array(['TJ 740', 'TJ 691', 'TSW 31', 'PPD 143', 'B 7353', 'KBM 01',
       'KBM 02', 'TJ 384', 'TJ 383', 'TJ 387', 'TJ 385', 'TJ 351',
       'TJ 884', 'TJ 354', 'TJ 353', 'TJ 350', 'B 7802', 'TJ 355',
       'TJ 348', 'TJ 555', 'TJ 545', 'MYS 21308', 'TJ 347', 'TJ 611',
       'PPD 654', 'TJ 869', 'MYS 19213', 'PPD 267', 'PPD 136', 'PPD 429',
       'KAJ 305', 'TJ 569', 'TJ 631', 'TJ 518', 'KMK 975'], dtype=object)

In [30]:
# realisasibus[realisasibus['No Body'].str.contains('')]
tob_5  = tob_4[(tob_4['No Body 2'].str.contains('H ')==False) & (tob_4['Kode Rute'].isna())]
tob_5.drop(['Kode Rute', 'No Body'], inplace=True, axis=1)

C:\Users\Nadya\AppData\Local\Temp\ipykernel_21168\1676331354.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tob_5.drop(['Kode Rute', 'No Body'], inplace=True, axis=1)


In [31]:
tob_5.groupby(['No Body 2', 'Day']).count().reset_index()['No Body 2'].unique()

array(['B 7353', 'B 7802', 'KAJ 305', 'KBM 01', 'KBM 02', 'KMK 975',
       'MYS 19213', 'MYS 21308', 'PPD 136', 'PPD 143', 'PPD 267',
       'PPD 429', 'PPD 654', 'TJ 347', 'TJ 348', 'TJ 350', 'TJ 351',
       'TJ 353', 'TJ 354', 'TJ 355', 'TJ 383', 'TJ 384', 'TJ 385',
       'TJ 387', 'TJ 518', 'TJ 545', 'TJ 555', 'TJ 569', 'TJ 611',
       'TJ 631', 'TJ 691', 'TJ 740', 'TJ 869', 'TJ 884', 'TSW 31'],
      dtype=object)

In [32]:
nopol = tob_5[tob_5['No Body 2'].str.contains('B ')]

In [33]:
# tob6 = tob_5.groupby(['No Body 2', 'Day']).count()['Hour'].reset_index()
tob_6 = tob_5[~tob_5['No Body 2'].str.contains('B ')]

#### Compile 

In [69]:
tob_4

,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,p_latitude_flo,p_longitude_flo,Waktu Tap,Hour,Day,Type,No Body 2,Tanggal,Kode Rute,No Body
0,H195,H195,47659,2023/7/17 00:00:06,9360885010145453,IN,-6.21361,107,2023-07-17 00:00:06,0,17,TOB,H 195,NaT,NaN,NaN
1,H204,H204,47668,2023/7/17 00:03:22,6032986090875120,IN,-6.25048,107,2023-07-17 00:03:22,0,17,TOB,H 204,NaT,NaN,NaN
2,H198,H198,47662,2023/7/17 00:03:50,6032982819048792,IN,-6.20164,107,2023-07-17 00:03:50,0,17,TOB,H 198,NaT,NaN,NaN
3,H198,H198,47662,2023/7/17 00:20:02,0145200029763838,IN,-6.20174,107,2023-07-17 00:20:02,0,17,TOB,H 198,NaT,NaN,NaN
4,H204,H204,47668,2023/7/17 00:20:13,9360885001900229,IN,-6.2505,107,2023-07-17 00:20:13,0,17,TOB,H 204,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391646,H190,H190,47654,2023/7/24 23:55:41,6032982610210898,OUT,-6.20174,107,2023-07-24 23:55:41,23,24,TOB,H 190,NaT,NaN,NaN
391647,H201,H201,47665,2023/7/24 23:56:08,6032981046426961,OUT,-6.25040,107,2023-07-24 23:56:08,23,24,TOB,H 201,NaT,NaN,NaN
391648,H201,H201,47665,2023/7/24 23:56:12,7546270009938742,OUT,-6.25040,107,2023-07-24 23:56:12,23,24,TOB,H 201,NaT,NaN,NaN
391649,H198,H198,47662,2023/7/24 23:56:20,6032982610210898,OUT,-6.20176,107,2023-07-24 23:56:20,23,24,TOB,H 198,NaT,NaN,NaN


In [70]:
T1 = tob_2[tob_2['Kode Rute'].isna()==False]
T2 = tob_4

TOB_X = pd.concat([T1, T2])
TOB_X
# TOB_X['idx_2'] = list(range(len(TOB_X))) 

In [92]:
TOB_X

,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,p_latitude_flo,p_longitude_flo,Waktu Tap,Hour,Day,Type,No Body 2,Kode Rute,No Bus,Tanggal,No Body,idx_2
40,TJ 802,B7166TGD,45318,2023/7/17 03:13:45,9360885001570617,IN,-6.31238,107,2023-07-17 03:13:45,3,17,TOB,TJ 802,14,TJ 0802,NaT,NaN,0
41,TJ 793,B7028TGD,47683,2023/7/17 03:19:29,9360885001570617,IN,-6.31247,107,2023-07-17 03:19:29,3,17,TOB,TJ 793,1H,TJ 0793,NaT,NaN,1
60,TJ 653,B7609TGC,45138,2023/7/17 04:37:51,0145200033965502,IN,-6.29129,107,2023-07-17 04:37:51,4,17,TOB,TJ 653,9H,TJ 0653,NaT,NaN,2
66,MYS 18177,B7842TGC,35516,2023/7/17 04:43:05,0145000132443561,IN,-2.54999995231628,118,2023-07-17 04:43:05,4,17,TOB,MYS 18177,9D,MYS 18177,NaT,NaN,3
67,MYS 18177,B7842TGC,35516,2023/7/17 04:43:11,0145200017117872,IN,-2.54999995231628,118,2023-07-17 04:43:11,4,17,TOB,MYS 18177,9D,MYS 18177,NaT,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391646,H190,H190,47654,2023/7/24 23:55:41,6032982610210898,OUT,-6.20174,107,2023-07-24 23:55:41,23,24,TOB,H 190,NaN,NaN,NaT,NaN,1992105
391647,H201,H201,47665,2023/7/24 23:56:08,6032981046426961,OUT,-6.25040,107,2023-07-24 23:56:08,23,24,TOB,H 201,NaN,NaN,NaT,NaN,1992106
391648,H201,H201,47665,2023/7/24 23:56:12,7546270009938742,OUT,-6.25040,107,2023-07-24 23:56:12,23,24,TOB,H 201,NaN,NaN,NaT,NaN,1992107
391649,H198,H198,47662,2023/7/24 23:56:20,6032982610210898,OUT,-6.20176,107,2023-07-24 23:56:20,23,24,TOB,H 198,NaN,NaN,NaT,NaN,1992108


#### c. BKO

In [93]:
tbko0 = pd.read_excel('../../Python/Data Transit Raw/Juli/Administrator  Transjakarta Transit 17-23 Juli BKO.xlsx', header=1)
tbko2 = pd.read_excel('../../Python/Data Transit Raw/Juli/Administrator  Transjakarta Transit 24 Juli BKO.xlsx', header=1)
tbko = pd.concat([tbko0, tbko2])

In [94]:
tbko ['Kode Rute'] = tbko ['Rute Real'].apply(lambda x: x.split('(')[-1].replace(')', ''))
tbko  = tbko .merge(kategori, on='Kode Rute', how='left')
tbko [tbko ['Jenis Layanan'].isna()]['Kode Rute'].unique()

array(['13E', '13F', 'L7', '5H', 'M5F'], dtype=object)

In [95]:
tbko.loc[tbko ['Kode Rute']=='M5F', 'Kode Rute']= '5F'
tbko.loc[tbko ['Kode Rute']=='5F', 'Jenis Layanan']= 'Non-BRT'

tbko = tbko.loc[tbko['Jenis Layanan']=='Non-BRT']

In [96]:
tbko['TimeStart'] = tbko['Tanggal Do'] +' '+ tbko['Jam Start Do']  
tbko['TimeEnd'] = tbko['Tanggal Do'] +' '+tbko['Jam End Do']  
tbko['TimeStart'] = tbko['TimeStart'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M')) 
tbko['Day'] = tbko['TimeStart'].apply(lambda x: x.day)
tbkox = tbko[tbko['TimeEnd'].isna()]
# dfta['TimeEnd'] = dfta['TimeEnd'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M')) 

In [97]:
tbkox

,Operator,No Bus,Nik,Pramudi,SPBU,Rute Real,Rute Renops,Rute Status,KM Start,KM End,KM Baku,Kategori DO,No Do,Tanggal Do,Jam Start Do,Jam End Do,Respons Verifikator,Respons Admin,Kode Rute,Jenis Layanan,Kategori Rute,TimeStart,TimeEnd,Day
3986,PPD 348,PPD 0123,23035,BERNARD SIHOTANG,NaN,STASIUN PALMERAH - TOSARI - STASIUN PALMERAH (1B),CIPUTAT - KEJAKSAAN AGUNG (S21),IncompleteLog0 / 33 Halte,523452,NaN,12,DO BKO,202307211121191556,2023-07-21,11:22,NaN,Proses,Proses,1B,Non-BRT,Angkutan Umum Integrasi,2023-07-21 11:22:00,NaN,21
5912,SWA MERCY 1526 MT,TJ 0250,108457,JUARI,NaN,STASIUN MANGGARAI - KUNINGAN TIMUR - STASIUN M...,STASIUN MANGGARAI - BLOK M (6M),IncompleteLog0 / 17 Halte,2657966,NaN,15,DO BKO,202307171119261919,2023-07-17,11:20,NaN,Proses,Proses,6M,Non-BRT,Angkutan Umum Integrasi,2023-07-17 11:20:00,NaN,17
5943,SWA MERCY 1626 AT Vintage,TJ 0472,119350,INDRA BUDI KURNIAWAN,NaN,STASIUN MANGGARAI - KUNINGAN TIMUR - STASIUN M...,STASIUN MANGGARAI - BLOK M (6M),IncompleteLog0 / 17 Halte,245841,NaN,15,DO BKO,202307211130141877,2023-07-21,11:30,NaN,Proses,Proses,6M,Non-BRT,Angkutan Umum Integrasi,2023-07-21 11:30:00,NaN,21
5964,SWA MERCY 1626 AT Vintage,TJ 0472,119350,INDRA BUDI KURNIAWAN,NaN,STASIUN MANGGARAI - KUNINGAN TIMUR - STASIUN M...,STASIUN MANGGARAI - BLOK M (6M),IncompleteLog0 / 17 Halte,245654,NaN,15,DO BKO,202307201050081159,2023-07-20,10:52,NaN,Proses,Proses,6M,Non-BRT,Angkutan Umum Integrasi,2023-07-20 10:52:00,NaN,20
5973,SWA MERCY 1626 AT Vintage,TJ 0472,119350,INDRA BUDI KURNIAWAN,NaN,STASIUN MANGGARAI - KUNINGAN TIMUR - STASIUN M...,STASIUN MANGGARAI - BLOK M (6M),IncompleteLog0 / 17 Halte,245502,NaN,15,DO BKO,202307181103461123,2023-07-18,11:04,NaN,Proses,Proses,6M,Non-BRT,Angkutan Umum Integrasi,2023-07-18 11:04:00,NaN,18
5993,SWA MERCY 1626 AT Vintage,TJ 0472,119350,INDRA BUDI KURNIAWAN,NaN,STASIUN MANGGARAI - KUNINGAN TIMUR - STASIUN M...,STASIUN MANGGARAI - BLOK M (6M),IncompleteLog0 / 17 Halte,2453358,NaN,15,DO BKO,202307171037021042,2023-07-17,10:39,NaN,Proses,Proses,6M,Non-BRT,Angkutan Umum Integrasi,2023-07-17 10:39:00,NaN,17
7089,PPD 348,PPD 0123,23032,ASEP TRIANA,NaN,STASIUN PALMERAH - TOSARI - STASIUN PALMERAH (1B),CIPUTAT - KEJAKSAAN AGUNG (S21),IncompleteLog0 / 33 Halte,524162,NaN,12,DO BKO,202307241223551401,2023-07-24,12:24,NaN,Proses,Proses,1B,Non-BRT,Angkutan Umum Integrasi,2023-07-24 12:24:00,NaN,24


In [98]:
dfto = dfta[dfta['TimeEnd'].isna()==False][['No Bus', 'Rute Real', 'TimeStart', 'Day', 'TimeEnd']]
dfto['TimeEnd'] = dfto['TimeEnd'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M'))  
dfto['Duration'] = dfto['TimeEnd'] - dfto['TimeStart']
dfto = dfto.groupby(['Rute Real', 'Day']).mean()[['Duration']].reset_index()

tbkox2 = tbkox.merge(dfto, on=['Rute Real', 'Day'], how='left')
tbkox2.loc[tbkox2['Duration'].isna(), 'Duration'] =tbkox2['Duration'].mean()
tbkox2['TimeEnd'] = tbkox2['TimeStart'] + tbkox2['Duration'] 

C:\Users\Nadya\AppData\Local\Temp\ipykernel_21168\4027859422.py:4: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  dfto = dfto.groupby(['Rute Real', 'Day']).mean()[['Duration']].reset_index()


In [99]:
tbko = tbko.dropna(subset='TimeEnd')
tbko['TimeEnd'] = tbko['TimeEnd'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M'))  
tbko2 = pd.concat([tbko, tbkox2])

In [100]:
tbko2 = tbko2[['No Bus', 'TimeStart','TimeEnd', 'Kode Rute', 'Day']]
tbko2['No Body 2'] = tbko2['No Bus'].apply (lambda x: x.replace(' ', ''))
tbko2['No Body 2'] = tbko2['No Body 2'].apply(lambda x: [re.findall(r'(\w+?)(\d+)', x.replace(' ', ''))][0][0][0] +' '+ [re.findall(r'(\w+?)(\d+)', x.replace(' ', ''))][0][0][1])
tbko2['No Body 2'] = tbko2['No Body 2'].apply(lambda x: x.upper().strip().replace(' 0', ' '))

In [71]:
tbko2.head(2)

,Operator,No Bus,Nik,Pramudi,SPBU,Rute Real,Rute Renops,Rute Status,KM Start,KM End,KM Baku,Kategori DO,No Do,Tanggal Do,Jam Start Do,Jam End Do,Respons Verifikator,Respons Admin
0,Bianglala Metropolitan (BMP AMARI),BMP 219,230485,BAMBANG HERMAWAN,NaN,TENDEAN - PURI BETA (13),TENDEAN - PURI BETA 2 (AMARI) (M13),IncompleteLog0 / 13 Halte,232181,232208,13,DO BKO,202307240237071094,2023-07-24,02:37,04:46,Proses,Proses
1,Bianglala Metropolitan (BMP AMARI),BMP 190,230617,RIZKY DWI NURKHOLIS FIRMANYSAH,NaN,TENDEAN - PURI BETA (13),PURI BETA 2 - TENDEAN (AMARI) (M13),IncompleteLog0 / 14 Halte,234894,234934,13,DO BKO,202307240145151396,2023-07-24,01:45,05:02,Proses,Proses


In [101]:
tobko = TOB_X.merge(tbko2, on=['No Body 2', 'Day'])
tobko = tobko[tobko['Waktu Tap'] > tobko['TimeStart']]
tobko = tobko[tobko['Waktu Tap'] < tobko['TimeEnd']]

In [102]:
TOB_X.head(1)

,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,p_latitude_flo,p_longitude_flo,Waktu Tap,Hour,Day,Type,No Body 2,Kode Rute,No Bus,Tanggal,No Body,idx_2
40,TJ 802,B7166TGD,45318,2023/7/17 03:13:45,9360885001570617,IN,-6.31238,107,2023-07-17 03:13:45,3,17,TOB,TJ 802,14,TJ 0802,NaT,NaN,0


In [103]:
tobko['Kode Rute_x'] = tobko['Kode Rute_y']
tobko.rename(columns={'Kode Rute_x': 'Kode Rute'}, inplace=True)
tobx2 = TOB_X[~TOB_X['idx_2'].isin(list(tobko['idx_2']))]
# tobx3 = pd.concat([tobx2, tobko])

In [90]:
tobko

,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,p_latitude_flo,p_longitude_flo,Waktu Tap,Hour,Day,Type,No Body 2,Kode Rute,No Bus_x,Tanggal,No Body,idx_2,No Bus_y,TimeStart,TimeEnd,Kode Rute_y,Kode Rute,Kode Rute
33,PPD 389,B7664TGB,35091,2023/7/17 19:13:43,6032982819944198,IN,-6.19028728688103,107,2023-07-17 19:13:43,19,17,TOB,PPD 389,T12,PPD 0389,NaT,NaN,122174,PPD 0389,2023-07-17 18:26:00,2023-07-17 20:17:00,T12,T12,T12
34,PPD 389,B7664TGB,35091,2023/7/17 19:15:19,0145000202963183,IN,-6.19019823070184,107,2023-07-17 19:15:19,19,17,TOB,PPD 389,T12,PPD 0389,NaT,NaN,122430,PPD 0389,2023-07-17 18:26:00,2023-07-17 20:17:00,T12,T12,T12
35,PPD 389,B7664TGB,35091,2023/7/17 19:15:23,6032982531707568,IN,-6.19018864535007,107,2023-07-17 19:15:23,19,17,TOB,PPD 389,T12,PPD 0389,NaT,NaN,122440,PPD 0389,2023-07-17 18:26:00,2023-07-17 20:17:00,T12,T12,T12
36,PPD 389,B7664TGB,35091,2023/7/17 19:15:34,6032984057577458,IN,-6.19024281855673,107,2023-07-17 19:15:34,19,17,TOB,PPD 389,T12,PPD 0389,NaT,NaN,122463,PPD 0389,2023-07-17 18:26:00,2023-07-17 20:17:00,T12,T12,T12
37,PPD 389,B7664TGB,35091,2023/7/17 19:15:44,0145200024698682,IN,-6.19024281855673,107,2023-07-17 19:15:44,19,17,TOB,PPD 389,T12,PPD 0389,NaT,NaN,122490,PPD 0389,2023-07-17 18:26:00,2023-07-17 20:17:00,T12,T12,T12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96947,PPD 112,B7406TGB,35021,2023/7/24 18:25:37,0145000700181767,IN,-6.20751766,107,2023-07-24 18:25:37,18,24,TOB,PPD 112,1B,NaN,2023-07-24,PPD 0112,1820309,PPD 0112,2023-07-24 17:44:00,2023-07-24 19:02:00,1B,1B,1B
96951,PPD 112,B7406TGB,35021,2023/7/24 18:34:30,6032982608426431,IN,-6.20751766,107,2023-07-24 18:34:30,18,24,TOB,PPD 112,1B,NaN,2023-07-24,PPD 0112,1820802,PPD 0112,2023-07-24 17:44:00,2023-07-24 19:02:00,1B,1B,1B
96966,PPD 112,B7406TGB,35021,2023/7/24 14:01:39,6032986059898824,OUT,-6.33260322,107,2023-07-24 14:01:39,14,24,TOB,PPD 112,1B,NaN,2023-07-24,PPD 0112,1981108,PPD 0112,2023-07-24 13:01:00,2023-07-24 14:21:00,1B,1B,1B
96970,PPD 112,B7406TGB,35021,2023/7/24 14:01:43,9360885004364955,OUT,-6.33260322,107,2023-07-24 14:01:43,14,24,TOB,PPD 112,1B,NaN,2023-07-24,PPD 0112,1981109,PPD 0112,2023-07-24 13:01:00,2023-07-24 14:21:00,1B,1B,1B


In [104]:
tobx3 = pd.concat([tobx2, tobko])

In [107]:
tobx3.drop(['idx_2', 'No Bus_x', 'No Bus_y', 'TimeStart', 'TimeEnd', 'Kode Rute_y'], axis=1, inplace=True)

In [ ]:
tobx3.head(2)

#### d. TOB Portable

In [108]:
tobp = pd.read_excel('../TOB Mobile gate.xlsx')
tobp['ID'] = tobp['ID'].apply(lambda x: x.replace(' ',''))

In [109]:
tobx3['ID'] = tobx3['shelter_name_var'].apply(lambda x: x.replace(' ','').replace('A', ''))
tobx4 = tobx3.merge(tobp, how='left', on='ID')
tobx4.loc[tobx4['Lokasi/ halte'].isna()==False, 'Type']='TOB Portable'

In [110]:
tobx4.loc[tobx4['Lokasi/ halte'].isna()==False, 'shelter_name_var']=tobx4['Lokasi/ halte']

In [111]:
len(tobx4), len(tobx4[tobx4['shelter_name_var'].isna()])

(1992110, 0)

In [112]:
print(len(tobx4.loc[(tobx4['Lokasi/ halte'].isna()) & (tobx4['Kode Rute'].isna())]['ID'].unique()))
tobx4.loc[(tobx4['Lokasi/ halte'].isna()) & (tobx4['Kode Rute'].isna())]['ID'].unique()

35


array(['TJ740', 'TJ691', 'TSW031', 'PPD143', 'B7353TGD', 'KBM001',
       'KBM002', 'TJ0384', 'TJ0383', 'TJ0387', 'TJ0385', 'TJ0351',
       'TJ0884', 'TJ0354', 'TJ0353', 'TJ0350', 'B7802SGX', 'TJ0355',
       'TJ0348', 'TJ555', 'TJ545', 'MYS21308', 'TJ0347', 'TJ611',
       'PPD654', 'TJ869', 'MYS19213', 'PPD267', 'PPD136', 'PPD429',
       'KJ305', 'TJ569', 'TJ631', 'TJ518', 'KMK975'], dtype=object)

In [113]:
tobx4.loc[(tobx4['Lokasi/ halte'].isna()) & (tobx4['Kode Rute'].isna()), 'Keterangan'] = 'Route Not Found'
tobx4.loc[tobx4['Type']=='TOB', 'shelter_code']= tobx4['shelter_name_var']
tobx4.loc[tobx4['Kode Rute'].isna()==False, 'shelter_name_var']= tobx4['Kode Rute']
tobx4.drop(['Kode Rute', 'Tanggal', 'No Body', 'ID', 'Lokasi/ halte', 'No Bus', 'No Body 2'], inplace=True, axis=1)

In [114]:
# tobx4.to_csv('../Juli IT/TOB processed.csv', index=False)

In [115]:
tobx4.sort_values(by=['card_number_var', 'Waktu Tap', 'tap'], ascending=True, inplace=True)

# tob5x apabila mau coba delete duplicate yg out saja tapi sepertinya tidak cocok
tob5x = tobx4.drop_duplicates(subset=['card_number_var', 'Waktu Tap'], keep='first')

In [116]:
tobx4[tobx4.duplicated(subset=['card_number_var', 'Waktu Tap'])]

,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,p_latitude_flo,p_longitude_flo,Waktu Tap,Hour,Day,Type,Keterangan
1062171,3E,TJ 257,46893,2023/7/18 17:39:47,0145000100246186,OUT,-6.15519,107,2023-07-18 17:39:47,17,18,TOB,NaN
1943889,Cibubur,H117,46944,2023/7/22 16:11:23,0145000100420724,OUT,-6.36882,107,2023-07-22 16:11:23,16,22,TOB Portable,NaN
1953293,Cibubur,H123,46950,2023/7/23 14:03:38,0145000100420724,OUT,-6.36852,107,2023-07-23 14:03:38,14,23,TOB Portable,NaN
1297873,7P,TSW 068,46298,2023/7/21 10:23:02,0145000100707963,OUT,-6.24781,107,2023-07-21 10:23:02,10,21,TOB,NaN
1042006,1N,TJ 499,45032,2023/7/18 14:52:26,0145000100757059,OUT,-6.24339,107,2023-07-18 14:52:26,14,18,TOB,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475352,6N,TSW 060,46282,2023/7/23 20:20:20,9360885010175014,OUT,-6.28351,107,2023-07-23 20:20:20,20,23,TOB,NaN
1449886,S21,PPD 446,46815,2023/7/23 12:32:40,9360885010176100,OUT,-6.25023,107,2023-07-23 12:32:40,12,23,TOB,NaN
936399,1N,TJ 828,45334,2023/7/17 16:04:36,9360885010200949,OUT,-6.18537,107,2023-07-17 16:04:36,16,17,TOB,NaN
1466862,6H,PPD 450,46817,2023/7/23 17:38:12,9360885010204810,OUT,-6.31185,107,2023-07-23 17:38:12,17,23,TOB,NaN


In [117]:

tob5x[tob5x['card_number_var']=='9360885010176100']

,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,p_latitude_flo,p_longitude_flo,Waktu Tap,Hour,Day,Type,Keterangan
691241,7B,MYS 21312,46359,2023/7/23 10:14:14,9360885010176100,IN,-6.30993,107,2023-07-23 10:14:14,10,23,TOB,NaN
700060,S21,PPD 446,46815,2023/7/23 12:32:40,9360885010176100,IN,-6.25023,107,2023-07-23 12:32:40,12,23,TOB,NaN
1453170,S21,PPD498,46835,2023/7/23 13:34:43,9360885010176100,OUT,-6.33107,107,2023-07-23 13:34:43,13,23,TOB,NaN


In [118]:
tobx4[tobx4['card_number_var']=='9360885010176100']

,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,p_latitude_flo,p_longitude_flo,Waktu Tap,Hour,Day,Type,Keterangan
691241,7B,MYS 21312,46359,2023/7/23 10:14:14,9360885010176100,IN,-6.30993,107,2023-07-23 10:14:14,10,23,TOB,NaN
700060,S21,PPD 446,46815,2023/7/23 12:32:40,9360885010176100,IN,-6.25023,107,2023-07-23 12:32:40,12,23,TOB,NaN
1449886,S21,PPD 446,46815,2023/7/23 12:32:40,9360885010176100,OUT,-6.25023,107,2023-07-23 12:32:40,12,23,TOB,NaN
1453170,S21,PPD498,46835,2023/7/23 13:34:43,9360885010176100,OUT,-6.33107,107,2023-07-23 13:34:43,13,23,TOB,NaN


## B. Compile In and Out

In [126]:
gate_in.head(1)

,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,Waktu Tap,Hour,Day,Type
0,ASEAN,1024,10080,2023/7/17 00:00:06,7546140002354405,IN,2023-07-17 00:00:06,0,17,GATE


In [127]:
tob_in.head(1)

,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,p_latitude_flo,p_longitude_flo,Waktu Tap,Hour,Day,Type
0,H195,H195,47659,2023/7/17 00:00:06,9360885010145453,IN,-6.21361,107,2023-07-17 00:00:06,0,17,TOB


In [130]:
tob_in2 = tobx4[tobx4['tap']=='IN']
tob_out2 = tobx4[tobx4['tap']=='OUT']

In [131]:
dfin = pd.concat([gate_in, tob_in2])
dfout = pd.concat([gate_out, tob_out2])

In [2]:
# dfin.to_csv('../Juli IT/In and Out/processed_in.csv')
# dfout.to_csv('../Juli IT/In and Out/processed_out.csv')

dfin = pd.read_csv('../Juli IT/In and Out/processed_in.csv')
dfout = pd.read_csv('../Juli IT/In and Out/processed_out.csv')

C:\Users\Nadya\AppData\Local\Temp\ipykernel_16332\3566996114.py:4: DtypeWarning: Columns (2,5,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  dfin =pd.read_csv('../Juli IT/In and Out/processed_in.csv')
C:\Users\Nadya\AppData\Local\Temp\ipykernel_16332\3566996114.py:5: DtypeWarning: Columns (2,5,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  dfout = pd.read_csv('../Juli IT/In and Out/processed_out.csv')


## C. Selecting the date

In [3]:
dfin0 = dfin[dfin['Day']==24]
dfout0 = dfout[dfout['Day']==24]

In [11]:
# dfin

In [4]:
dfin0['idx_in'] = range(len(dfin0))
dfout0['idx_out'] = range(len(dfout0))

# dfin['waktu_tap'] = dfin['waktu_tap'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')) 
# dfout['waktu_tap'] = dfout['waktu_tap'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')) 

# dfin['card_number_var'] = dfin['card_number_var'].apply(lambda x: str(x))
# dfout['card_number_var'] = dfout['card_number_var'].apply(lambda x: str(x))

dfin2 = dfin0[['idx_in', 'Waktu Tap', 'card_number_var']]
dfout2 = dfout0[['idx_out','Waktu Tap', 'card_number_var']]

len(dfin2), len(dfout2)

C:\Users\Nadya\AppData\Local\Temp\ipykernel_16332\713918214.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfin0['idx_in'] = range(len(dfin0))
C:\Users\Nadya\AppData\Local\Temp\ipykernel_16332\713918214.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfout0['idx_out'] = range(len(dfout0))


(542703, 492805)

In [5]:
len(dfin2['idx_in'].unique()), len(dfout2['idx_out'].unique())

(542703, 492805)

In [15]:
dfin

,Unnamed: 0,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,Waktu Tap,Hour,Day,Type,p_latitude_flo,p_longitude_flo,Keterangan
0,0,ASEAN,1024,10080,2023/7/17 00:00:06,7546140002354405,IN,2023-07-17 00:00:06,0,17,GATE,NaN,NaN,NaN
1,1,PRJ,12224,10891,2023/7/17 00:00:08,7546270000983465,IN,2023-07-17 00:00:08,0,17,GATE,NaN,NaN,NaN
2,2,CAWANG SOETOYO,10176,10451,2023/7/17 00:00:15,145200301278422,IN,2023-07-17 00:00:15,0,17,GATE,NaN,NaN,NaN
3,3,KALI BESAR BARAT,12216,10558,2023/7/17 00:00:24,9360885009429755,IN,2023-07-17 00:00:24,0,17,GATE,NaN,NaN,NaN
4,4,LEBAK BULUS,8121,10353,2023/7/17 00:00:30,9360885000028391,IN,2023-07-17 00:00:30,0,17,GATE,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3596278,215047,7B,PPD 186,35243,2023/7/18 15:22:53,F8143148A8BCAA9B,IN,2023-07-18 15:22:53,15,18,TOB,-2.54999995231628,118,NaN
3596279,303119,7B,PPD 175,35233,2023/7/19 14:05:17,F8143148A8BCAA9B,IN,2023-07-19 14:05:17,14,19,TOB,-6.2001412,107,NaN
3596280,395368,6N,TSW 088,35375,2023/7/20 12:25:14,FAA80FAC284CD811,IN,2023-07-20 12:25:14,12,20,TOB,-6.17056696,107,NaN
3596281,1703271,KBM002,KBM002,35666,2023/7/20 13:08:38,FAA80FAC284CD811,IN,2023-07-20 13:08:38,13,20,TOB,-2.54999995231628,118,Route Not Found


# 2. Outer Merging Process

### a. Merge Outer and Find Duation

In [6]:
dfm1 = dfin2.merge(dfout2, on='card_number_var', how='left')

In [7]:
dfm1['duration'] = dfm1['Waktu Tap_y'] - dfm1['Waktu Tap_x']
dfm1['duration'] = dfm1['duration'].apply(lambda x: x.total_seconds()/60)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [1248]:
# dfin2[dfin2['Card Number Var']==9360885002017024]

In [1249]:
# dfout2[dfout2['Card Number Var']==9360885002017024]

### b. Filter: Duration > 0

In [143]:
dfm2 = dfm1[dfm1['duration']>0.07]

print(len(dfm1), len(dfm2))

981292 694413


### c. Filter: Reasonable Duration
1. Filter the matched dataframe with duration under 6 hours so that the extreme values won't drag the mean and standar deviation up
2. Filter based on mean and standard deviation

In [144]:
dfm3 = dfm2[dfm2['duration'] < 360 ]

In [149]:
dfm4 = dfm3[(dfm3['duration'] < dfm3['duration'].mean() + (4 * dfm3['duration'].std()))]
print(dfm4['duration'].mean(), dfm4['duration'].std(), len(dfm4))
dfm4

54.77012276150285 51.452456714467445 530025


,idx_in,Waktu Tap_x,card_number_var,idx_out,Waktu Tap_y,duration
0,0,2023-07-24 00:00:05,7546270010095565,716,2023-07-24 00:29:37,30
2,1,2023-07-24 00:00:06,6032981050756394,407946,2023-07-24 00:52:56,53
3,2,2023-07-24 00:00:07,0145200011969880,870,2023-07-24 00:34:59,35
5,3,2023-07-24 00:00:07,0145000624311474,534,2023-07-24 00:18:45,19
6,4,2023-07-24 00:00:11,6032982837685294,438730,2023-07-24 00:52:45,53
...,...,...,...,...,...,...
981283,542698,2023-07-24 14:55:18,9360885010210057,202126,2023-07-24 16:04:14,69
981285,542699,2023-07-24 05:48:48,9360885010210133,17594,2023-07-24 06:32:11,43
981288,542700,2023-07-24 10:53:14,9360885010211090,149080,2023-07-24 11:42:09,49
981289,542700,2023-07-24 10:53:14,9360885010211090,492802,2023-07-24 10:55:20,2


In [1254]:
# dfm4['duration'].plot(kind='kde')

In [150]:
dfm4.boxplot(column='duration')

<AxesSubplot:>

### d. Filter: Delete Duplicated Tap In/Out

In [151]:
dfm4 = dfm4.sort_values(by='duration', ascending=True)
dfm4 = dfm4.drop_duplicates(subset='idx_out', keep='first')
dfm4 = dfm4.drop_duplicates(subset='idx_in', keep='first')
print (len(dfm4))
dfm4

465455


,idx_in,Waktu Tap_x,card_number_var,idx_out,Waktu Tap_y,duration
947460,526531,2023-07-24 19:16:09,7546270006380666,480095,2023-07-24 19:16:14,0
872660,484918,2023-07-24 06:01:00,6032984007863875,446369,2023-07-24 06:01:05,0
872670,484924,2023-07-24 11:36:34,6032984007891934,446376,2023-07-24 11:36:39,0
783091,435975,2023-07-24 17:24:04,6032981030613624,406416,2023-07-24 17:24:09,0
783075,435964,2023-07-24 09:52:46,6032981030524375,406410,2023-07-24 09:52:51,0
...,...,...,...,...,...,...
824394,458027,2023-07-24 07:28:45,6032982609694482,424221,2023-07-24 12:32:42,304
775761,432008,2023-07-24 11:47:14,6013500456754382,403169,2023-07-24 16:51:27,304
668826,373205,2023-07-24 09:18:05,0145000132819752,356052,2023-07-24 14:22:25,304
702845,391753,2023-07-24 07:32:58,0145007400440724,370942,2023-07-24 12:37:57,305


### e. Sampling Check

In [152]:
cardnum = dfm4.iloc[-19000]['card_number_var']

In [153]:
dfin2[dfin2['card_number_var']==cardnum]

,idx_in,Waktu Tap,card_number_var
2198736,72833,2023-07-24 07:22:53,0145000617697673
2358876,232973,2023-07-24 16:29:39,0145000617697673
750292,381630,2023-07-24 06:46:30,0145000617697673


In [154]:
dfout2[dfout2['card_number_var']==cardnum]

,idx_out,Waktu Tap,card_number_var
2255528,226215,2023-07-24 17:12:57,0145000617697673
1512643,362881,2023-07-24 09:06:57,0145000617697673


In [155]:
dfm4[dfm4['card_number_var']==cardnum]

,idx_in,Waktu Tap_x,card_number_var,idx_out,Waktu Tap_y,duration
418441,232973,2023-07-24 16:29:39,0145000617697673,226215,2023-07-24 17:12:57,43
129400,72833,2023-07-24 07:22:53,0145000617697673,362881,2023-07-24 09:06:57,104


In [1260]:
dfm4

,idx_in,Waktu Tap_x,card_number_var,idx_out,Waktu Tap_y,duration
101321,49809,2023-07-23 19:06:40,9360885001723903,41055,2023-07-23 19:06:44,0
48421,23570,2023-07-23 12:33:00,9360885001723056,18954,2023-07-23 12:33:04,0
48424,23571,2023-07-23 12:33:00,0145200100252347,18955,2023-07-23 12:33:04,0
64882,31568,2023-07-23 14:34:42,0145200010736066,25245,2023-07-23 14:34:46,0
48426,23572,2023-07-23 12:33:02,6032982600953069,18957,2023-07-23 12:33:06,0
...,...,...,...,...,...,...
147390,72739,2023-07-23 03:04:08,6032982833789330,1183,2023-07-23 06:19:55,196
147158,72604,2023-07-23 02:24:36,1003062206494442,48640,2023-07-23 05:40:28,196
53646,26057,2023-07-23 13:10:05,6032982606174488,55730,2023-07-23 16:26:09,196
5278,2557,2023-07-23 06:31:11,6032984078057266,101932,2023-07-23 09:47:26,196


# 3. Merge Back and Retreive DF with Full Information

### a. Merge the Paired and the Unpaired

In [156]:
paired_in = list(dfm3['idx_in'].unique())
paired_out = list(dfm3['idx_out'].unique())

unpaired_in = dfin2[~dfin2['idx_in'].isin(paired_in)]
unpaired_out = dfout2[~dfout2['idx_out'].isin(paired_out)]

In [157]:
dfm5 = pd.concat([dfm4, unpaired_in, unpaired_out])
dfm5

,idx_in,Waktu Tap_x,card_number_var,idx_out,Waktu Tap_y,duration,Waktu Tap
947460,526531,2023-07-24 19:16:09,7546270006380666,480095,2023-07-24 19:16:14,0,NaT
872660,484918,2023-07-24 06:01:00,6032984007863875,446369,2023-07-24 06:01:05,0,NaT
872670,484924,2023-07-24 11:36:34,6032984007891934,446376,2023-07-24 11:36:39,0,NaT
783091,435975,2023-07-24 17:24:04,6032981030613624,406416,2023-07-24 17:24:09,0,NaT
783075,435964,2023-07-24 09:52:46,6032981030524375,406410,2023-07-24 09:52:51,0,NaT
...,...,...,...,...,...,...,...
1540019,NaN,NaT,9360885010145417,492751,NaT,NaN,2023-07-24 15:25:23
1978784,NaN,NaT,9360885010145434,492752,NaT,NaN,2023-07-24 18:27:44
1523182,NaN,NaT,9360885010161973,492772,NaT,NaN,2023-07-24 11:03:59
1548063,NaN,NaT,9360885010163068,492773,NaT,NaN,2023-07-24 16:36:58


In [158]:
dfm5.loc[(dfm5['idx_out'].isna()) & (dfm5['idx_in'].isna()==False), 'Waktu Tap_x'] =dfm5.loc[(dfm5['idx_out'].isna()) & (dfm5['idx_in'].isna()==False)]['Waktu Tap']

dfm5.loc[(dfm5['idx_in'].isna()) & (dfm5['idx_out'].isna()==False), 'Waktu Tap_y'] =dfm5.loc[(dfm5['idx_in'].isna()) & (dfm5['idx_out'].isna()==False)]['Waktu Tap']

In [159]:
dfm5.drop('Waktu Tap', inplace=True, axis=1)

In [160]:
print(f'% Tap In dan Tap Out yang berhasil di-paired: {round(len(dfm4)/len(dfm5)*100, 2)} % ') 

% Tap In dan Tap Out yang berhasil di-paired: 83.12 % 


### b. Merge with Original Dataframe

In [1266]:
# dfin

In [161]:
dfm6 = dfm5[['idx_in', 'idx_out']]
dfm7 = dfm6.merge(dfin0, on='idx_in', how='left')
dfm8 = dfm7.merge(dfout0, on='idx_out', how='left')

In [162]:
dfm8[(dfm8['Waktu Tap_x'].isna()==False) & (dfm8['Waktu Tap_y'].isna()==False)].head(2)

,idx_in,idx_out,shelter_name_var_x,shelter_code_x,terminal_code_x,waktu_tap_x,card_number_var_x,tap_x,Waktu Tap_x,Hour_x,Day_x,Type_x,p_latitude_flo_x,p_longitude_flo_x,Keterangan_x,shelter_name_var_y,shelter_code_y,terminal_code_y,waktu_tap_y,card_number_var_y,tap_y,Waktu Tap_y,Hour_y,Day_y,Type_y,p_latitude_flo_y,p_longitude_flo_y,Keterangan_y
0,526531,480095,6T,TJ 650,25364,2023/7/24 19:16:09,7546270006380666,IN,2023-07-24 19:16:09,19,24,TOB,0.000000,0,NaN,6T,TJ 650,25364,2023/7/24 19:16:14,7546270006380666,OUT,2023-07-24 19:16:14,19,24,TOB,0.000000,0,NaN
1,484918,446369,1A,MYS 18192,35522,2023/7/24 06:01:00,6032984007863875,IN,2023-07-24 06:01:00,6,24,TOB,-6.31737704,107,NaN,1A,MYS 18192,35522,2023/7/24 06:01:05,6032984007863875,OUT,2023-07-24 06:01:05,6,24,TOB,-6.31737704,107,NaN


In [163]:
# dfin0

In [164]:
dfm8.loc[dfm8['p_latitude_flo_x'].isna()==False, 'coordinates_in'] = dfm8[dfm8['p_latitude_flo_x'].isna()==False]['p_latitude_flo_x'].apply(lambda x: str(x)) + ', ' + dfm8[dfm8['p_latitude_flo_x'].isna()==False]['p_latitude_flo_x'].apply(lambda x: str(x))
dfm8.loc[dfm8['p_latitude_flo_y'].isna()==False, 'coordinates_out'] = dfm8[dfm8['p_latitude_flo_y'].isna()==False]['p_latitude_flo_y'].apply(lambda x: str(x)) + ', ' + dfm8[dfm8['p_latitude_flo_y'].isna()==False]['p_latitude_flo_y'].apply(lambda x: str(x))

In [165]:
dfm8.rename(columns={'shelter_name_var_x' : 'shelter_name_in', 'Waktu Tap_x': 'waktu_tap_in', 'shelter_name_var_y': 'shelter_name_out', 
                     'Waktu Tap_y': 'waktu_tap_out', 'terminal_code_x':'terminal_code_in', 'terminal_code_y':'terminal_code_out',
                     'Type_x': 'Type_in', 'Type_y': 'Type_out', 'Day_x':'Hari', 'card_number_var_x': 'card_number_var',
                    'shelter_code_x': 'shelter_code_in', 'shelter_code_y': 'shelter_code_out'}, inplace=True)

In [166]:
dfm8['card_number_var'] = dfm8['card_number_var'].fillna(dfm8['card_number_var_y'])

In [167]:
dfm8[dfm8['idx_in'].isna()].head(8)

,idx_in,idx_out,shelter_name_in,shelter_code_in,terminal_code_in,waktu_tap_x,card_number_var,tap_x,waktu_tap_in,Hour_x,Hari,Type_in,p_latitude_flo_x,p_longitude_flo_x,Keterangan_x,shelter_name_out,shelter_code_out,terminal_code_out,waktu_tap_y,card_number_var_y,tap_y,waktu_tap_out,Hour_y,Day_y,Type_out,p_latitude_flo_y,p_longitude_flo_y,Keterangan_y,coordinates_in,coordinates_out
534302,NaN,0,NaN,NaN,NaN,NaN,6013500438007875,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,KAMPUNG SUMUR,11200,10870,2023/7/24 00:00:04,6013500438007875,OUT,2023-07-24 00:00:04,0,24,GATE,NaN,NaN,NaN,NaN,NaN
534303,NaN,1,NaN,NaN,NaN,NaN,6032982609000367,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,SEMANGGI,9161,10830,2023/7/24 00:00:04,6032982609000367,OUT,2023-07-24 00:00:04,0,24,GATE,NaN,NaN,NaN,NaN,NaN
534304,NaN,2,NaN,NaN,NaN,NaN,6032984010834632,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,SWADHARMA,13239,10588,2023/7/24 00:00:04,6032984010834632,OUT,2023-07-24 00:00:04,0,24,GATE,NaN,NaN,NaN,NaN,NaN
534305,NaN,3,NaN,NaN,NaN,NaN,6013500456695247,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,KAMPUNG SUMUR,11200,10512,2023/7/24 00:00:06,6013500456695247,OUT,2023-07-24 00:00:06,0,24,GATE,NaN,NaN,NaN,NaN,NaN
534306,NaN,4,NaN,NaN,NaN,NaN,6032981032540668,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,TEGALAN,5077,10206,2023/7/24 00:00:10,6032981032540668,OUT,2023-07-24 00:00:10,0,24,GATE,NaN,NaN,NaN,NaN,NaN
534307,NaN,5,NaN,NaN,NaN,NaN,7546270009614210,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,INDOSIAR,3051,10113,2023/7/24 00:00:11,7546270009614210,OUT,2023-07-24 00:00:11,0,24,GATE,NaN,NaN,NaN,NaN,NaN
534308,NaN,6,NaN,NaN,NaN,NaN,0145200500437175,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,PEJATEN,6094,10790,2023/7/24 00:00:24,0145200500437175,OUT,2023-07-24 00:00:24,0,24,GATE,NaN,NaN,NaN,NaN,NaN
534309,NaN,7,NaN,NaN,NaN,NaN,7546000017002797,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,DUKUH ATAS 1,1009,10032,2023/7/24 00:00:26,7546000017002797,OUT,2023-07-24 00:00:26,0,24,GATE,NaN,NaN,NaN,NaN,NaN


In [168]:
dfm9 = dfm8[['idx_in', 'idx_out',  'card_number_var',  'shelter_name_in', 'shelter_code_in',
       'terminal_code_in', 'waktu_tap_in',  'Type_in','shelter_name_out', 'shelter_code_out','terminal_code_out',
       'waktu_tap_out',  'Hari', 'Type_out', 'coordinates_in', 'coordinates_out']]

In [169]:
# dfm8.loc[(dfm8['coordinates_in'] != dfm8['coordinates_out']) & (dfm8['coordinates_out'].isna()==False) &( dfm8['coordinates_in'].isna()==False) ]

In [170]:
# dfm9[dfm9['card_number_var']=='9360885002017023']

In [171]:
dfm9.iloc[[109000,150000]]

,idx_in,idx_out,card_number_var,shelter_name_in,shelter_code_in,terminal_code_in,waktu_tap_in,Type_in,shelter_name_out,shelter_code_out,terminal_code_out,waktu_tap_out,Hari,Type_out,coordinates_in,coordinates_out
109000,336330,311121,0145200025765175,SAWAH BESAR ARAH BLOK M,1029,10989,2023-07-24 19:25:41,GATE,BUNDARAN HI,1011,10916,2023-07-24 19:44:53,24,GATE,NaN,NaN
150000,400362,377891,0145200013853165,4F,TJ 738,35741,2023-07-24 15:18:11,TOB,4F,TJ 738,35741,2023-07-24 15:43:31,24,TOB,"-2.549999952316284, -2.549999952316284","-2.549999952316284, -2.549999952316284"


### c. Export

In [172]:
Day = str(round(dfm9['Hari'].iloc[0]))
dfm9.to_csv(f'../Juli IT/matched_again/20230731_{Day}_Juni 2023_First Matched.csv', index=False)

# 4 Match with Koridor

## EXPLORE DUPLICATES

In [1173]:
dfm9

,idx_in,idx_out,card_number_var,shelter_name_in,shelter_code_in,terminal_code_in,waktu_tap_in,Type_in,shelter_name_out,shelter_code_out,terminal_code_out,waktu_tap_out,Hari,Type_out,coordinates_in,coordinates_out
0,19131,11224,0145200002793877,7A,MYS 22362,35625,2023-07-24 06:55:45,TOB,7A,MYS 22362,35625,2023-07-24 06:55:49,24,TOB,"-6.34382053, -6.34382053","-6.34382053, -6.34382053"
1,43093,30748,7546150005414486,5M,TJ 846,35827,2023-07-24 08:50:15,TOB,5M,TJ 846,35827,2023-07-24 08:50:19,24,TOB,"-6.31264954, -6.31264954","-6.31264954, -6.31264954"
2,43035,30701,9360885001832573,5M,TJ 846,35827,2023-07-24 08:49:56,TOB,5M,TJ 846,35827,2023-07-24 08:50:00,24,TOB,"-6.31264954, -6.31264954","-6.31264954, -6.31264954"
3,121951,95034,0145000701906956,SH1,TJ 837,47176,2023-07-24 19:06:09,TOB,SH1,TJ 837,47176,2023-07-24 19:06:13,24,TOB,"-6.15382, -6.15382","-6.15382, -6.15382"
4,42947,30633,0145200300919232,7A,MYS 22356,35601,2023-07-24 08:49:25,TOB,7A,MYS 22356,35601,2023-07-24 08:49:29,24,TOB,"-6.3096027, -6.3096027","-6.3096027, -6.3096027"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565355,NaN,491362,0145200016437412,NaN,NaN,NaN,NaT,NaN,PERMAI KOJA,10191,10471,2023-07-24 23:15:53,NaN,GATE,NaN,NaN
565356,NaN,491364,6013500139962410,NaN,NaN,NaN,NaT,NaN,PERMAI KOJA,10191,10466,2023-07-24 23:15:54,NaN,GATE,NaN,NaN
565357,NaN,491977,6032982602704312,NaN,NaN,NaN,NaT,NaN,PGC2,7222,10976,2023-07-24 23:30:33,NaN,GATE,NaN,NaN
565358,NaN,492041,9360885001555991,NaN,NaN,NaN,NaT,NaN,TANAH KUSIR KODIM,8125,10357,2023-07-24 23:32:40,NaN,GATE,NaN,NaN


In [759]:
100 - (len(dfm9[dfm9['idx_in'].isna()]) + len(dfm9[dfm9['idx_out'].isna()]) + len(dfm9[dfm9['waktu_tap_in'] == dfm9['waktu_tap_out']])) / len(dfm9)*100

81.5977193584153

In [582]:
len(dfm9[dfm9['waktu_tap_in'] == dfm9['waktu_tap_out']])

17361

In [615]:
# tob['card_number_var'].iloc[1000]

In [589]:
# tob.sort_values(by=['card_number_var', 'waktu_tap', 'tap']).drop_duplicates()
tob.drop_duplicates(subset=['card_number_var', 'waktu_tap'], keep='first')

,shelter_name_var,shelter_code,terminal_code,waktu_tap,card_number_var,tap,p_latitude_flo,p_longitude_flo,Waktu Tap,Hour,Day,Type,No Body 2
0,H195,H195,47659,2023/7/17 00:00:06,9360885010145453,IN,-6.21361,107,2023-07-17 00:00:06,0,17,TOB,H 195
1,H204,H204,47668,2023/7/17 00:03:22,6032986090875120,IN,-6.25048,107,2023-07-17 00:03:22,0,17,TOB,H 204
2,H198,H198,47662,2023/7/17 00:03:50,6032982819048792,IN,-6.20164,107,2023-07-17 00:03:50,0,17,TOB,H 198
3,H198,H198,47662,2023/7/17 00:20:02,0145200029763838,IN,-6.20174,107,2023-07-17 00:20:02,0,17,TOB,H 198
4,H204,H204,47668,2023/7/17 00:20:13,9360885001900229,IN,-6.2505,107,2023-07-17 00:20:13,0,17,TOB,H 204
...,...,...,...,...,...,...,...,...,...,...,...,...,...
891821,H187,H187,47651,2023/7/24 23:50:44,0145200302001450,OUT,-6.17468,107,2023-07-24 23:50:44,23,24,TOB,H 187
891822,H190,H190,47654,2023/7/24 23:55:41,6032982610210898,OUT,-6.20174,107,2023-07-24 23:55:41,23,24,TOB,H 190
891824,H201,H201,47665,2023/7/24 23:56:12,7546270009938742,OUT,-6.25040,107,2023-07-24 23:56:12,23,24,TOB,H 201
891825,H198,H198,47662,2023/7/24 23:56:20,6032982610210898,OUT,-6.20176,107,2023-07-24 23:56:20,23,24,TOB,H 198


In [604]:
# len(gate) - len(gate.drop_duplicates(subset=['card_number_var', 'waktu_tap'], keep='first'))

# tob[tob.duplicated(subset=['card_number_var', 'waktu_tap'], keep=False)].sort_values(by=['card_number_var', 'waktu_tap']).to_csv('../Juli IT/Duplicated to the seconds.csv')

In [610]:
len(tob[tob.duplicated(subset=['card_number_var', 'waktu_tap'], keep=False)].sort_values(by=['card_number_var', 'waktu_tap']))/2

108335.5

# 4. Pelanggan AMARI

In [1080]:
entries = os.listdir('D:\Transjakarta\Pelanggan\Tap In Out Juni IT\Output\second try')
df0 = pd.DataFrame()

for i in entries: 
    df = pd.read_csv(f'output/second try/{i}', dtype={'card_number_var':str})
    df0 = pd.concat([df0, df])
    

'2023-06-19 14:17:26'

In [1082]:
# df0 ['waktu_tap_in'] =df0 ['waktu_tap_in'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')) 

df0.loc[df0['waktu_tap_in'].isna()==False, 'waktu_tap_in'] = df0[df0['waktu_tap_in'].isna()==False]['waktu_tap_in'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df0.loc[df0['waktu_tap_out'].isna()==False, 'waktu_tap_out'] = df0[df0['waktu_tap_out'].isna()==False]['waktu_tap_out'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [1083]:
dfa_in = df0.loc[df0['idx_out'].isna()]
dfa_out = df0.loc[df0['idx_in'].isna()]

len(dfa_in), len(dfa_out)

(469185, 115518)

In [1084]:
dfa.columns
dfa_in = dfa_in[['idx_in',  'card_number_var', 'shelter_name_in',
       'shelter_code_in', 'terminal_code_in', 'waktu_tap_in', 'Type_in', 'coordinates_in']]

dfa_out = dfa_out[['idx_out', 'card_number_var',  'shelter_name_out', 'shelter_code_out', 'terminal_code_out',
       'waktu_tap_out', 'Hari', 'Type_out', 'coordinates_out' ]]



In [1085]:
len(dfa_in), len(dfa_out)

(469185, 115518)

In [28]:
# dfa_in.dropna(subset='idx_in', inplace=True)
# dfa_out.dropna(subset='idx_out', inplace=True)

C:\Users\Nadya\AppData\Local\Temp\ipykernel_7660\4101481305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa_in.dropna(subset='idx_in', inplace=True)
C:\Users\Nadya\AppData\Local\Temp\ipykernel_7660\4101481305.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa_out.dropna(subset='idx_out', inplace=True)


In [166]:
dfa_in

,idx_in,card_number_var,shelter_name_in,shelter_code_in,terminal_code_in,waktu_tap_in,Type_in,coordinates_in
453775,1,6032984043928997,PURI BETA 2,13243.0,10904,2023-06-19 07:18:53,GATE,NaN
453776,15,145200003329853,CIKOKO ST.CAWANG ARAH P.RANTI,9148.0,10898,2023-06-19 07:18:55,GATE,NaN
453777,46,145002501791950,RAWA BUAYA,3046.0,10745,2023-06-19 07:18:57,GATE,NaN
453778,84,145200032650402,KAMPUNG RAMBUTAN,7120.0,10314,2023-06-19 07:19:00,GATE,NaN
453779,92,145000149218287,PENJARINGAN,9174.0,10435,2023-06-19 07:19:01,GATE,NaN
...,...,...,...,...,...,...,...,...
296775,297324,7546270003134470,TSW 100,B7580TAA,35399,2023-06-25 11:18:09,TOB,"-6.17059977, 106.88582057"
296776,297325,145200006226619,TSW 100,B7580TAA,35399,2023-06-25 11:23:12,TOB,"-6.17059977, 106.88582057"
296777,297326,145000139407510,TSW 100,B7580TAA,35399,2023-06-25 11:58:16,TOB,"-6.17059977, 106.88582057"
296778,297332,7546270010127665,TSW 100,B7580TAA,35399,2023-06-25 12:16:36,TOB,"-6.17059977, 106.88582057"


In [441]:
# dfa_in['waktu_tap_in'] = dfa_in['waktu_tap_in'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S')) 
# dfa_out['waktu_tap_out'] = dfa_out['waktu_tap_out'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S')) 

In [1086]:
dfa_in['jam_tap'] = dfa_in['waktu_tap_in'].apply(lambda x: x.hour)
dfa_out['jam_tap'] = dfa_out['waktu_tap_out'].apply(lambda x: x.hour)

In [1087]:
dfa_in = dfa_in[dfa_in['jam_tap']>21]
dfa_out = dfa_out[dfa_out['jam_tap']>21]

In [1088]:
len(dfa_in), len(dfa_out)

(14859, 1208)

In [1089]:
dfa2 = dfa_in.merge(dfa_out, on= 'card_number_var', how='left')

In [447]:
# dfa_in['card_number_var'].iloc[0]
# dfa_out['card_number_var'].iloc[0]

In [187]:
# dfa_out

In [1095]:
dfa2['duration'] = dfa2 ['waktu_tap_out'] - dfa2 ['waktu_tap_in']
dfa2['duration'] = dfa2['duration'].apply(lambda x: x.total_seconds()/60)
dfa2

,idx_in,card_number_var,shelter_name_in,shelter_code_in,terminal_code_in,waktu_tap_in,Type_in,coordinates_in,jam_tap_x,idx_out,shelter_name_out,shelter_code_out,terminal_code_out,waktu_tap_out,Hari,Type_out,coordinates_out,jam_tap_y,duration
0,344,7546270002530686,ADAM MALIK,13241.0,10592,2023-06-19 23:19:05,GATE,NaN,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,345,6032983200083414,ADAM MALIK,13241.0,10592,2023-06-19 23:19:29,GATE,NaN,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37913,1003072102794706,ANCOL,5089.0,10953,2023-06-19 22:00:15,GATE,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38037,6013500167619601,ASEAN,1024.0,10080,2023-06-19 22:37:25,GATE,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,38042,145015300066366,ASEAN,1024.0,10080,2023-06-19 22:55:05,GATE,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14888,294040,145000155414820,TSW 075,B7557TAA,35349,2023-06-25 22:17:38,TOB,"-2.549999952, 118.010002136",22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14889,294042,9360885010046739,TSW 075,B7557TAA,35349,2023-06-25 22:38:54,TOB,"-2.549999952, 118.010002136",22,272524,H207,H207,47671,2023-06-25 22:00:08,NaN,TOB,"-6.2278, 106.80058",22,-39
14890,295548,6032982609751449,TSW 087,B7579TAA,35372,2023-06-25 22:03:24,TOB,"-6.17040587, 106.88570674",22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14891,295549,6013500606347335,TSW 087,B7579TAA,35372,2023-06-25 22:03:30,TOB,"-6.17040587, 106.88570674",22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1096]:
dfa3 = dfa2[dfa2['duration']>=0]

print(len(dfa2), len(dfa3))

14893 69


In [1097]:
dfa4 = dfa3.sort_values(by='duration', ascending=True)
dfa4 = dfa3.drop_duplicates(subset='idx_out', keep='first')
dfa5 = dfa4.drop_duplicates(subset='idx_in', keep='first')
print (len(dfa5))
dfa5

49


,idx_in,card_number_var,shelter_name_in,shelter_code_in,terminal_code_in,waktu_tap_in,Type_in,coordinates_in,jam_tap_x,idx_out,shelter_name_out,shelter_code_out,terminal_code_out,waktu_tap_out,Hari,Type_out,coordinates_out,jam_tap_y,duration
45,41645,6032984027261159,BENDUNGAN HILIR,1006.0,10029,2023-06-19 23:02:21,GATE,NaN,23,371914,MYS 21287,B7563TGD,46334,2023-06-20 22:25:27,NaN,TOB,"-6.21318, 106.8197",22,1403
693,143852,7546270009577250,LATUMENTEN ST. K.A ARAH PLUIT,9169.0,10834,2023-06-19 23:11:13,GATE,NaN,23,411153,MYS 21319,B7598TGD,46366,2023-06-23 22:35:35,NaN,TOB,"-6.36854, 106.89394",22,5724
861,166737,6032982514109055,PENJARINGAN,9174.0,10435,2023-06-19 23:00:29,GATE,NaN,23,219939,MATRAMAN I,5078.0,10773,2023-06-23 22:08:53,NaN,GATE,NaN,22,5708
1557,403642,6032984087229351,H189,H189,47653,2023-06-19 22:03:27,TOB,"-6.22788, 106.80074",22,345221,PEMUDA RW.MANGUN,4060.0,10157,2023-06-20 23:10:04,NaN,GATE,NaN,23,1507
1560,403652,145000138915281,H189,H189,47653,2023-06-19 23:16:32,TOB,"-6.2279, 106.80075",23,256960,H194,H194,47658,2023-06-24 23:12:46,NaN,TOB,"-6.22783, 106.80067",23,7196
1601,407296,145200005272796,H209,H209,47673,2023-06-19 23:21:50,TOB,"-6.22778, 106.80063",23,349727,H182,H182,47646,2023-06-20 23:56:34,NaN,TOB,"-6.20722, 106.84171",23,1475
1603,407297,145000147571612,H209,H209,47673,2023-06-19 23:21:57,TOB,"-6.22778, 106.80063",23,397719,H180,H180,47644,2023-06-22 23:58:52,NaN,TOB,"-6.20719, 106.84169",23,4357
1604,407341,7546270010258510,H209,H209,47673,2023-06-19 23:13:57,TOB,"-6.22783, 106.80066",23,352159,H207,H207,47671,2023-06-20 23:13:14,NaN,TOB,"-6.22785, 106.80064",23,1439
1606,407640,145000625284340,H209,H209,47673,2023-06-19 22:08:02,TOB,"-6.22774, 106.8007",22,352348,H207,H207,47671,2023-06-20 22:12:36,NaN,TOB,"-6.22799, 106.80069",22,1445
1611,418261,6013500159592527,KBM220006,B7916TAA,35674,2023-06-19 22:06:13,TOB,"-6.23781558, 106.75417187",22,414743,KBM220007,B7914TAA,35677,2023-06-22 22:01:19,NaN,TOB,"-6.2377935, 106.75409681",22,4315


In [1099]:
df0[df0['card_number_var']=='145200308996794']

,idx_in,idx_out,card_number_var,shelter_name_in,shelter_code_in,terminal_code_in,waktu_tap_in,Type_in,shelter_name_out,shelter_code_out,terminal_code_out,waktu_tap_out,Hari,Type_out,coordinates_in,coordinates_out
395081,453626,88422,145200308996794,PPD 302,B7513TGB,46784,2023-06-19 22:16:28,TOB,BUNDARAN HI,1011.0,10951,2023-06-19 23:37:49,19,GATE,"-6.09362, 106.75054",NaN
432243,117142,463782,145200308996794,MASJID AGUNG,1002.0,10014,2023-06-19 08:54:57,GATE,PPD 448,B7596TGB,46816,2023-06-19 10:41:54,19,TOB,NaN,"-6.13314, 106.80618"
428611,438121,164496,145200308996794,PPD 408,B7737TGB,46805,2023-06-21 22:10:48,TOB,MASJID AGUNG,1002.0,10014,2023-06-21 23:59:46,21,GATE,"-6.10868, 106.73972",NaN
473904,475988,153744,145200308996794,PPD 233,B7568TGB,35148,2023-06-22 19:22:52,TOB,MASJID AGUNG,1002.0,10014,2023-06-22 20:57:39,22,GATE,"-6.108461972, 106.739808032",NaN
522550,180272,NaN,145200308996794,MASJID AGUNG,1002.0,10014,2023-06-22 08:04:13,GATE,NaN,NaN,NaN,NaN,22,NaN,NaN,NaN
576219,530562,NaN,145200308996794,PPD 419,B7712TGB,35294,2023-06-22 09:54:37,TOB,NaN,NaN,NaN,NaN,22,NaN,"-6.112130564, 106.742601721",NaN
371790,492006,91053,145200308996794,PPD 303,B7506TGB,35184,2023-06-23 19:13:14,TOB,BUNDARAN HI,1011.0,10916,2023-06-23 20:20:44,23,GATE,"-6.18033162, 106.82870413",NaN
425849,92502,405459,145200308996794,MASJID AGUNG,1002.0,10014,2023-06-23 07:33:40,GATE,PPD 472,B7720TGB,46825,2023-06-23 09:08:54,23,TOB,NaN,"0.0, 0.0"
285809,73387,275282,145200308996794,MASJID AGUNG,1002.0,10014,2023-06-24 10:25:34,GATE,PPD 428,B7787TGB,46809,2023-06-24 12:05:43,24,TOB,NaN,"-6.11319, 106.79462"
344763,317190,NaN,145200308996794,PPD 429,B7723TGB,35130,2023-06-24 22:18:49,TOB,NaN,NaN,NaN,NaN,24,NaN,"-6.1082277, 106.7400788",NaN


# Compile All

In [5]:
entries = os.listdir('D:\Transjakarta\Pelanggan\Juli IT\output')
df0 = pd.DataFrame()

for i in entries: 
    df = pd.read_csv(f'../Juli IT/output/{i}', dtype={'card_number_var':str})
    df0 = pd.concat([df0, df])
    
df0.loc[df0['waktu_tap_in'].isna()==False, 'waktu_tap_in'] = df0[df0['waktu_tap_in'].isna()==False]['waktu_tap_in'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df0.loc[df0['waktu_tap_out'].isna()==False, 'waktu_tap_out'] = df0[df0['waktu_tap_out'].isna()==False]['waktu_tap_out'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

C:\Users\Nadya\AppData\Local\Temp\ipykernel_21604\728562453.py:5: DtypeWarning: Columns (3,4,6,7,8,9,11,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'../Juli IT/output/{i}', dtype={'card_number_var':str})
C:\Users\Nadya\AppData\Local\Temp\ipykernel_21604\728562453.py:5: DtypeWarning: Columns (3,4,6,7,8,9,11,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'../Juli IT/output/{i}', dtype={'card_number_var':str})
C:\Users\Nadya\AppData\Local\Temp\ipykernel_21604\728562453.py:5: DtypeWarning: Columns (4,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'../Juli IT/output/{i}', dtype={'card_number_var':str})
C:\Users\Nadya\AppData\Local\Temp\ipykernel_21604\728562453.py:5: DtypeWarning: Columns (3,4,6,7,8,9,11,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'../Juli IT/output/

## Get the Corridor

In [17]:
dfhk = pd.read_excel('halte matched.xlsx')

In [1284]:
# import difflib
# dhalte2['Halte 2']=dhalte2['HALTE'].apply(lambda x: difflib.get_close_matches(x, dfhk['HALTE'])) 
# dhalte2['Halte 3'] = dhalte2['Halte 2'].apply(lambda x: x[0] if len(x)>0 else np.nan)

In [26]:
# dfm2[dfm2['Type_out']=='GATE']

In [8]:
dfm1 = df0.merge(dfhk, left_on='shelter_name_in', right_on='HALTE', how='left')
dfm2 = dfm1.merge(dfhk, left_on='shelter_name_out', right_on='HALTE', how='left')
dfm2 = dfm2.rename(columns={'KORIDOR_x': 'Kor_in', 'KORIDOR_y': 'Kor_out'})
dfm2.drop([])

len(df0), len(dfm1), len(dfm2)

(3736733, 3736733, 3736733)

In [9]:
dfm2.drop(['HALTE_x',	'HALTE 2_x', 	'HALTE_y',	'HALTE 2_y'], axis=1, inplace=True)

In [10]:
dfm2.loc[(dfm2['shelter_name_in'].isna()), 'Ket 2'] = 'In Not Found'
dfm2.loc[(dfm2['shelter_name_out'].isna()), 'Ket 2'] = 'Out Not Found'
dfm2.loc[(dfm2['shelter_name_out'].isna()==False) & (dfm2['shelter_name_in'].isna()==False), 'Ket 2'] = 'Complete'

In [12]:
# dfm2.loc[dfm2['waktu_tap_in'].isna()==False, 'waktu_tap_in'] = dfm2[dfm2['waktu_tap_in'].isna()==False]['waktu_tap_in'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
# dfm2.loc[dfm2['waktu_tap_out'].isna()==False, 'waktu_tap_out'] = dfm2[dfm2['waktu_tap_out'].isna()==False]['waktu_tap_out'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [14]:
# dfm2['waktu_tap_in'].iloc[0]

In [31]:
dfm2.loc[dfm2['waktu_tap_in'].isna()==False, 'Hari'] = dfm2[dfm2['waktu_tap_in'].isna()==False]['waktu_tap_in'].apply(lambda x: x.day)
dfm2.loc[dfm2['waktu_tap_out'].isna()==False, 'Hari'] = dfm2[dfm2['waktu_tap_out'].isna()==False]['waktu_tap_out'].apply(lambda x: x.day)

In [33]:
dfm2['Hari'].isna().sum()

0

### Clear Up the TOP Portable Names (again)

In [3]:
# dfm2= pd.read_csv('../Juli IT/output/Matched all_0823.csv', dtype={'card_number_var':str})

NameError: name 'pd' is not defined

In [35]:
dfm2[dfm2['Type_in']=='TOB Portable']['shelter_name_in'].unique()

array(['Slipi petamburan arah pluit', 'Slipi petamburan arah p.ranti',
       'Bunderan Senayan arah Kota', 'KIP / PTC', 'Cibubur', 'St Tebet',
       'Bunderan Senayan arah Blok M', 'Stasiun Tanah Abang', 'Buperta',
       'St gondangdia', 'Juanda', 'pengajuan tjbrt', 'Kalideres',
       'St Tanjung barat', 'Pancoran Barat arah Pluit', 'kebon jeruk',
       'Pulomas arah pulogadung 1', 'Cawang Uki Arah kp melayu 2',
       'Karet sudirman arah kota', 'Cawang Uki Arah kp rambutan 1',
       'Karet sudirman arah blokm', 'Cawang Uki Arah kp melayu 1',
       'Karet sudirman arah blok m', 'St Palmerah',
       'Pasar rumput arah tosari', 'Pulomas arah kwitang 2',
       'Cawang Uki Arah kp rambutan 2', 'Pasar rumput arah pulogadung',
       'Sunter Kelapa Gading', 'Pancoran Barat arah P.ranti',
       'Gelora Bung Karno', 'Tosari', 'Bank Indonesia arah Blok M', '9E',
       'Pulomas arah pulogadung 2', 'Petojo', 'PIK/1A',
       'Jembatan baru arah kota', 'Masjid Agung', 'Puri Beta 2'],
 

In [40]:
portable = pd.read_excel('../TOB Mobile gate.xlsx')
portable.head(2)

,ID,Lokasi/ halte
0,H089,Stasiun Tanjung barat
1,H090,Stasiun Tanjung barat


In [41]:
portable['Lokasi/ halte'].unique()

array(['Stasiun Tanjung barat', 'Masjid Agung', 'Kebon jeruk',
       'Stasiun Tanah Abang', 'Kalideres', 'kebon jeruk', 'Tosari',
       'Ancol', 'Freshmarket PIK', 'Petojo', 'Stasiun Palmerah',
       'Stasiun gondangdia', 'Cibubur', 'Puri Beta 2', 'KIP',
       'Sunter Kelapa Gading', 'Juanda', 'PIK Avenue', 'Karet',
       'Stasiun Tebet', 'Stasiun Kalibata', 'Kebon Jeruk',
       'Karet Kuningan', 'Stasiun Gondangdia',
       'S Parman Podomoro arah pluit', 'Bank Indonesia arah Blok M',
       'kantor', 'Pancoran Barat 1', 'Bank Indonesia arah Kota',
       'Stasiunasiun Tanah Abang', 'Semanggi', 'Pancoran Barat 2',
       'Pulomas arah pulogadung 1', 'Pulomas arah pulogadung 2',
       'Jembatan baru arah blokm', 'Jembatan baru arah kota',
       'Pasar rumput arah tosari', 'Pasar rumput arah pulogadung',
       'Pancoran Barat arah Pluit', 'Pulomas arah kwitang 1',
       'Slipi petamburan arah p.ranti', 'Pulomas arah kwitang 2',
       'Bunderan Senayan arah Kota', 'Slipi petam

In [42]:
dfm3 = dfm2.merge(portable, left_on='shelter_code_in', right_on='ID', how='left')
dfm4 = dfm3.merge(portable, left_on='shelter_code_out', right_on='ID', how='left')

len(dfm2), len(dfm4)

(3736733, 3736733)

In [43]:
# dfm4.loc[dfm4['Lokasi/ halte_x'].isna()==False]

,idx_in,idx_out,card_number_var,shelter_name_in,shelter_code_in,terminal_code_in,waktu_tap_in,Type_in,shelter_name_out,shelter_code_out,terminal_code_out,waktu_tap_out,Hari,Type_out,coordinates_in,coordinates_out,Kor_in,Kor_out,Ket 2,ID_x,Lokasi/ halte_x,ID_y,Lokasi/ halte_y
26,139139,113455,9360885002397268,Slipi petamburan arah pluit,H198,47662,2023-07-17 06:15:56,TOB Portable,Slipi petamburan arah pluit,H198,47662,2023-07-17 06:16:00,17,TOB Portable,"-6.20164, -6.20164","-6.20164, -6.20164",NaN,NaN,Complete,H198,Slipi petamburan arah pluit,H198,Slipi petamburan arah pluit
28,160905,129048,0145200401285038,Slipi petamburan arah pluit,H198,47662,2023-07-17 16:28:24,TOB Portable,Slipi petamburan arah pluit,H198,47662,2023-07-17 16:28:28,17,TOB Portable,"-6.20174, -6.20174","-6.20174, -6.20174",NaN,NaN,Complete,H198,Slipi petamburan arah pluit,H198,Slipi petamburan arah pluit
40,161268,129345,7546270004369299,Slipi petamburan arah p.ranti,H186,47650,2023-07-17 16:39:16,TOB Portable,Slipi petamburan arah p.ranti,H186,47650,2023-07-17 16:39:20,17,TOB Portable,"-6.20107, -6.20107","-6.20107, -6.20107",NaN,NaN,Complete,H186,Slipi petamburan arah p.ranti,H186,Slipi petamburan arah p.ranti
44,161289,129359,7546270008042736,Slipi petamburan arah pluit,H198,47662,2023-07-17 16:39:42,TOB Portable,Slipi petamburan arah pluit,H198,47662,2023-07-17 16:39:46,17,TOB Portable,"-6.20162, -6.20162","-6.20162, -6.20162",NaN,NaN,Complete,H198,Slipi petamburan arah pluit,H198,Slipi petamburan arah pluit
52,138765,113296,7546270006653286,Slipi petamburan arah pluit,H198,47662,2023-07-17 06:10:51,TOB Portable,Slipi petamburan arah pluit,H198,47662,2023-07-17 06:10:55,17,TOB Portable,"-6.20162, -6.20162","-6.20162, -6.20162",NaN,NaN,Complete,H198,Slipi petamburan arah pluit,H198,Slipi petamburan arah pluit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3683643,171835,NaN,6032982610165969,Bunderan Senayan arah Blok M,H193,47657,2023-07-24 23:42:28,TOB Portable,NaN,NaN,NaN,NaN,24,NaN,"-6.22752, -6.22752",NaN,NaN,NaN,Out Not Found,H193,Bunderan Senayan arah Blok M,NaN,NaN
3683644,171836,NaN,6032982500492952,Cawang Uki Arah kp melayu 1,H206,47670,2023-07-24 23:45:02,TOB Portable,NaN,NaN,NaN,NaN,24,NaN,"-6.24971, -6.24971",NaN,NaN,NaN,Out Not Found,H206,Cawang Uki Arah kp melayu 1,NaN,NaN
3683645,171837,NaN,0145000132945714,Karet sudirman arah kota,H209,47673,2023-07-24 23:48:30,TOB Portable,NaN,NaN,NaN,NaN,24,NaN,"-6.21362, -6.21362",NaN,NaN,NaN,Out Not Found,H209,Karet sudirman arah kota,NaN,NaN
3683646,171838,NaN,6032982600971574,Bunderan Senayan arah Kota,H189,47653,2023-07-24 23:49:20,TOB Portable,NaN,NaN,NaN,NaN,24,NaN,"-6.22787, -6.22787",NaN,NaN,NaN,Out Not Found,H189,Bunderan Senayan arah Kota,NaN,NaN


In [44]:
dfm4.loc[(dfm4['Lokasi/ halte_x'].isna()==False) & (dfm4['Type_in']=='TOB Portable'), 'shelter_name_in'] = dfm4.loc[dfm4['Lokasi/ halte_x'].isna()==False]['Lokasi/ halte_x']
dfm4.loc[(dfm4['Lokasi/ halte_y'].isna()==False) & (dfm4['Type_out']=='TOB Portable'), 'shelter_name_out'] = dfm4.loc[dfm4['Lokasi/ halte_y'].isna()==False]['Lokasi/ halte_y']

In [45]:
dfm4.drop(['Lokasi/ halte_x', 'Lokasi/ halte_y', 'ID_x', 'ID_y' ], axis=1, inplace=True)

In [ ]:
# dfm4.loc[dfm4['Hari'].isna(), 'Hari'] = dfm4.loc[dfm4['Hari'].isna()]['waktu_tap_out'].apply(lambda x: x.day)

In [46]:
dfm4.loc[dfm4['shelter_name_in'].isna()==False, 'shelter_name_in'] = dfm4.loc[dfm4['shelter_name_in'].isna()==False]['shelter_name_in'].apply(lambda x: x.strip().upper())
dfm4.loc[dfm4['shelter_name_out'].isna()==False, 'shelter_name_out'] = dfm4.loc[dfm4['shelter_name_out'].isna()==False]['shelter_name_out'].apply(lambda x: x.strip().upper())

In [49]:
dfm4.loc[dfm4['Type_in']=='TOB']['shelter_name_in'].unique()

array(['6H', '8E', '5M', '8D', '1E', 'D31', '8ST', '1P', '1K', '7A', '1T',
       '6D', '1N', 'B11', '6U', '4F', '1H', '6P', '10K', '1A', '11Q',
       '3E', 'S31', 'SH1', 'KBM002', '10ST', 'S12', 'R1A', '5B', '8M',
       'JIS3', '8C', '2B', 'B21', '11D', 'TJ0383', 'KBM001', 'D32',
       'TJ 740', '7B', '5N', 'TJ0387', '12F', '9D', '4C', 'S11', 'S22',
       '9H', '6T', '9E', '7P', '12B', '1U', 'T11', 'TJ0385', '6M', '1Q',
       '5F', '7D', '7ST', '6K', '1B', '6N', '12P', 'B13', 'B14', 'TJ0384',
       '1F', '4B', '12H', '1M', 'D21', '7Q', '1C', '14', '2Q', '1R', '7E',
       '6C', 'S21', 'D11', '7C', '8K', '1', '6Q', 'TSW 031', '2P', '11M',
       '2F', 'TJ 691', '12A', 'T12', 'PPD 143', 'H097', '11K', '5',
       'B7353TGD', '9F', '10A', '7', '3C', '10D', '9', '3', 'B7802SGX',
       'TJ0350', 'TJ0355', 'TJ0351', 'TJ0354', 'TJ0348', 'TJ0884',
       'TJ0353', 'TJ 545', 'TJ 555', '7F', 'BW2', 'BW4', 'BW9',
       'MYS 21308', '2', '12', 'TJ0347', 'TJ 611', '7V', 'TGS108', '6A',
   

In [50]:
# dfm4.to_csv('../Juli IT/output2/230803_Matched all_2.csv', index=False)

In [1]:
# dfm4

NameError: name 'dfm4' is not defined

### Compile Lagi

In [19]:
dfm2.columns

dfin = dfm2[['idx_in','card_number_var', 'shelter_name_in', 'shelter_code_in', 'terminal_code_in', 'waktu_tap_in',
             'Type_in', 'Hari', 'Kor_in']]
dfout = dfm2[['idx_out', 'card_number_var',  'shelter_name_out', 'shelter_code_out', 'terminal_code_out',
       'waktu_tap_out', 'Type_out','Hari',  'Kor_out']]

In [30]:
dfin.dropna(subset='shelter_name_in',  inplace=True)
dfout.dropna(subset='shelter_name_out', inplace=True)

dfin['tap'] ='in'
dfout['tap'] ='out'

C:\Users\Nadya\AppData\Local\Temp\ipykernel_3908\2002439606.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfin.dropna(subset='shelter_name_in',  inplace=True)
C:\Users\Nadya\AppData\Local\Temp\ipykernel_3908\2002439606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfout.dropna(subset='shelter_name_out', inplace=True)
C:\Users\Nadya\AppData\Local\Temp\ipykernel_3908\2002439606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [47]:
dfm2[dfm2['Hari'].isna()]['Ket 2'].unique()

array(['In Not Found'], dtype=object)

In [34]:
dfin.columns
dfin.rename( columns={'shelter_name_in': 'shelter_name', 'shelter_code_in':'shelter_code',
       'terminal_code_in': 'terminal_code', 'waktu_tap_in':'waktu_tap', 'Type_in':'Type', 
              'Kor_in':'Kor'}, inplace=True)

dfout.rename( columns={'shelter_name_out': 'shelter_name', 'shelter_code_out':'shelter_code',
       'terminal_code_out': 'terminal_code', 'waktu_tap_out':'waktu_tap', 'Type_out':'Type', 
              'Kor_out':'Kor'}, inplace=True)

C:\Users\Nadya\AppData\Local\Temp\ipykernel_3908\3753460975.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfin.rename( columns={'shelter_name_in': 'shelter_name', 'shelter_code_in':'shelter_code',
C:\Users\Nadya\AppData\Local\Temp\ipykernel_3908\3753460975.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfout.rename( columns={'shelter_name_out': 'shelter_name', 'shelter_code_out':'shelter_code',


In [36]:
# alltap = pd.concat([dfin, dfout])
# alltap.to_csv('../Juli IT/all tap.csv')

In [39]:
alltap[alltap['card_number_var']=='0145200013060167'].sort_values(by='waktu_tap', ascending=True)

,idx_in,card_number_var,shelter_name,shelter_code,terminal_code,waktu_tap,Type,Hari,Kor,tap,idx_out
351376,34,0145200013060167,B21,PPD 633,46849,2023-07-17 04:55:22,TOB,17,NaN,in,NaN
542164,NaN,0145200013060167,B21,PPD 633,46849,2023-07-17 04:55:22,TOB,NaN,NaN,out,7
468628,6023,0145200013060167,9D,MYS 21232,45762,2023-07-17 05:54:12,TOB,17,NaN,in,NaN
351376,NaN,0145200013060167,9D,MYS 21232,45762,2023-07-17 05:54:12,TOB,17,NaN,out,2765
484006,71080,0145200013060167,7D,BHL220509,42455,2023-07-17 14:09:13,TOB,17,NaN,in,NaN
547875,NaN,0145200013060167,7D,BHL220509,42455,2023-07-17 14:09:13,TOB,NaN,NaN,out,55044
484577,73684,0145200013060167,B21,PPD 365,35282,2023-07-17 14:38:46,TOB,17,NaN,in,NaN
1086019,230606,0145200013060167,BULAK KAPAL,14226,10602,2023-07-18 07:00:49,GATE,18,B21,in,NaN
1046270,46739,0145200013060167,9D,MYS 18177,35516,2023-07-18 09:23:58,TOB,18,NaN,in,NaN
2460444,70449,0145200013060167,7D,PPD 307,35270,2023-07-21 14:26:36,TOB,21,NaN,in,NaN


In [50]:
alltap[alltap['card_number_var']=='6032982849090483'].sort_values(by='waktu_tap', ascending=True)

,idx_in,card_number_var,shelter_name,shelter_code,terminal_code,waktu_tap,Type,Hari,Kor,tap,idx_out
542160,NaN,6032982849090483,2B,PPD 433,46810,2023-07-17 04:52:26,TOB,NaN,NaN,out,2
308452,351037,6032982849090483,ASMI,2024,10056,2023-07-17 13:24:09,GATE,17,KORIDOR 2,in,NaN
308452,NaN,6032982849090483,2B,PPD 201,46750,2023-07-17 14:13:42,TOB,17,NaN,out,55340
2207632,3827,6032982849090483,2B,PPD 314,35188,2023-07-21 05:43:08,TOB,21,NaN,in,NaN
2207632,NaN,6032982849090483,ASMI,2024.0,10722,2023-07-21 06:18:42,GATE,21,KORIDOR 2,out,147855
2387002,333429,6032982849090483,ASMI,2024,10056,2023-07-21 13:02:05,GATE,21,KORIDOR 2,in,NaN
2387002,NaN,6032982849090483,ANCOL,5089.0,10785,2023-07-21 14:17:17,GATE,21,KORIDOR 5,out,306511
2442480,451045,6032982849090483,ANCOL,5089,10221,2023-07-21 17:56:50,GATE,21,KORIDOR 5,in,NaN
2442480,NaN,6032982849090483,2B,PPD 433,46810,2023-07-21 20:24:21,TOB,21,NaN,out,100789
3542334,3386,6032982849090483,2B,PPD 261,35202,2023-07-24 05:35:51,TOB,24,NaN,in,NaN


In [1324]:
# dfx1 = dfm2[(dfm2['Type_in']=='GATE') & (dfm2['Type_out']=='GATE')]
# dfx2 = dfm2[(dfm2['Tyxpe_in']=='GATE') & (dfm2['Type_out']=='TOB Portable')]
# dfx3 = dfm2[(dfm2['Type_in']=='TOB Portable') & (dfm2['Type_out']=='GATE')]

# dfx = pd.concat([dfx1, dfx2, dfx3])

# 5. Explore

In [127]:
# dfm4.drop('Unnamed: 0', axis=1, inplace=True)
dfm4.head(1)

,idx_in,idx_out,card_number_var,shelter_name_in,shelter_code_in,terminal_code_in,waktu_tap_in,Type_in,shelter_name_out,shelter_code_out,terminal_code_out,waktu_tap_out,Hari,Type_out,coordinates_in,coordinates_out,Kor_in,Kor_out,Ket 2
0,127845,102812,0145200025413503,6H,PPD 464,35304,2023-07-17 20:01:27,TOB,6H,PPD 464,35304,2023-07-17 20:01:31,17,TOB,"-2.54999995231628, -2.54999995231628","-2.54999995231628, -2.54999995231628",NaN,NaN,Complete


### Extract only O-D Pairs

In [156]:
OD1 = dfm4[(dfm4['Type_in']=='GATE') & (dfm4['Type_out']=='GATE')]
OD2 = dfm4[(dfm4['Type_in']=='TOB Portable') & (dfm4['Type_out']=='GATE')]
OD3 = dfm4[(dfm4['Type_in']=='GATE') & (dfm4['Type_out']=='TOB Portable')]
OD4 = dfm4[(dfm4['Type_in']=='TOB Portable') & (dfm4['Type_out']=='TOB Portable')]

OD0 = pd.concat([OD1, OD2, OD3, OD4])

In [152]:
# dfm4.loc[(dfm4['Type_in']=='TOB Portable') & ( dfm4['shelter_name_in']=='9E')] = 'Stasiun Palmerah'
# dfm4.loc[(dfm4['Type_out']=='TOB Portable') & ( dfm4['shelter_name_out']=='9E'), 'shelter_name_out'] = 'Stasiun Palmerah'

In [164]:
OD = OD0.groupby(['shelter_name_in', 'shelter_name_out']).count()[['Kor_in']].reset_index()
OD = OD[['shelter_name_in', 'shelter_name_out']]
OD = OD[OD['shelter_name_in'] != OD['shelter_name_out']]

In [166]:
OD['shelter_name_in']  = OD['shelter_name_in'].apply(lambda x: x.strip().upper())
OD['shelter_name_out']  = OD['shelter_name_out'].apply(lambda x: x.strip().upper())

In [168]:
# OD.to_excel('../Juli IT/OD.xlsx')

In [128]:
dfm4.loc[dfm4['Hari'].isna(), 'Hari'] = dfm4.loc[dfm4['Hari'].isna()]['waktu_tap_out'].apply(lambda x: x.day)

In [131]:
df17 = dfm4[dfm4['Hari']==17]

In [138]:
df17[(df17['Kor_in']=='KORIDOR 13') & (df17['Kor_out']=='KORIDOR 1') & (df17['shelter_name_in']!='CSW') & (df17['shelter_name_in']!='BLOK M')]

,idx_in,idx_out,card_number_var,shelter_name_in,shelter_code_in,terminal_code_in,waktu_tap_in,Type_in,shelter_name_out,shelter_code_out,terminal_code_out,waktu_tap_out,Hari,Type_out,coordinates_in,coordinates_out,Kor_in,Kor_out,Ket 2
41335,372600,326669,6032982606735692,KEJAKSAAN AGUNG,1023,10083,2023-07-17 15:00:56,GATE,BLOK M,1001.0,10702,2023-07-17 15:07:09,17,GATE,NaN,NaN,KORIDOR 13,KORIDOR 1,Complete
41417,323942,281692,6032984077206468,ASEAN,1024,10080,2023-07-17 11:09:21,GATE,MASJID AGUNG,1002.0,10014,2023-07-17 11:15:35,17,GATE,NaN,NaN,KORIDOR 13,KORIDOR 1,Complete
48623,366880,322647,6032984081341079,KEJAKSAAN AGUNG,1023,10084,2023-07-17 14:38:48,GATE,BLOK M,1001.0,10702,2023-07-17 14:46:38,17,GATE,NaN,NaN,KORIDOR 13,KORIDOR 1,Complete
51685,326169,284340,9360885001554056,ASEAN,1024,10080,2023-07-17 11:20:47,GATE,MASJID AGUNG,1002.0,10014,2023-07-17 11:29:15,17,GATE,NaN,NaN,KORIDOR 13,KORIDOR 1,Complete
52368,409198,354520,6032982526758824,KEJAKSAAN AGUNG,1023,10084,2023-07-17 16:34:21,GATE,MASJID AGUNG,1002.0,10014,2023-07-17 16:42:58,17,GATE,NaN,NaN,KORIDOR 13,KORIDOR 1,Complete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467223,328232,317380,6013500440117589,PURI BETA 2,13243,10904,2023-07-17 11:31:33,GATE,BLOK M,1001.0,10701,2023-07-17 14:18:43,17,GATE,NaN,NaN,KORIDOR 13,KORIDOR 1,Complete
467411,205569,247882,6032984082927322,PURI BETA 1,13242,10595,2023-07-17 06:25:22,GATE,BLOK M,1001.0,10702,2023-07-17 09:16:40,17,GATE,NaN,NaN,KORIDOR 13,KORIDOR 1,Complete
467484,246031,269221,6032984006988889,PURI BETA 2,13243,10905,2023-07-17 07:25:27,GATE,BLOK M,1001.0,10702,2023-07-17 10:18:05,17,GATE,NaN,NaN,KORIDOR 13,KORIDOR 1,Complete
467538,306621,298319,9360885003494884,PURI BETA 1,13242,10595,2023-07-17 09:47:29,GATE,BLOK M,1001.0,10701,2023-07-17 12:41:34,17,GATE,NaN,NaN,KORIDOR 13,KORIDOR 1,Complete


In [136]:
df17['Kor_in'].unique()

array([nan, 'KORIDOR 5', 'KORIDOR 1', 'KORIDOR 7', 'KORIDOR 2',
       'KORIDOR 3', 'KORIDOR 11', 'KORIDOR 9', 'KORIDOR 13', 'KORIDOR 10',
       'KORIDOR 6', 'KORIDOR 8', 'KORIDOR 4', 'KORIDOR 12', 'B11', 'B21'],
      dtype=object)

In [1326]:
dfx.groupby(['shelter_name_in', 'shelter_name_out']).count()

idx_in  idx_out  card_number_var  \
shelter_name_in shelter_name_out                                          
9E              BERMIS                      1        1                1   
                DISPENDA                   11       11               11   
                DURI KEPA                   3        3                3   
                GN.SAHARI MG.DUA            2        2                2   
                INDOSIAR                   22       22               22   
...                                       ...      ...              ...   
pengajuan tjbrt SAWAH BESAR ARAH KOTA       1        1                1   
                SESKOAL                     2        2                2   
                SETIABUDI UTARA AINI        4        4                4   
                VELBAK                      1        1                1   
                WARUNG JATI                 2        2                2   

                                       shelter_code_in  terminal_code_in  \
shelter_name_in shelter_name_out                                           
9E              BERMIS                               1                 1   
                DISPENDA                            11                11   
                DURI KEPA                            3                 3   
                GN.SAHARI MG.DUA                     2                 2   
                INDOSIAR                            22                22   
...                                                ...               ...   
pengajuan tjbrt SAWAH BESAR ARAH KOTA                1                 1   
                SESKOAL                              2                 2   
                SETIABUDI UTARA AINI                 4                 4   
                VELBAK                               1                 1   
                WARUNG JATI                          2                 2   

                                       waktu_tap_in  Type_in  \
shelter_name_in shelter_name_out                               
9E              BERMIS                            1        1   
                DISPENDA                         11       11   
                DURI KEPA                         3        3   
                GN.SAHARI MG.DUA                  2        2   
                INDOSIAR                         22       22   
...                                             ...      ...   
pengajuan tjbrt SAWAH BESAR ARAH KOTA             1        1   
                SESKOAL                           2        2   
                SETIABUDI UTARA AINI              4        4   
                VELBAK                            1        1   
                WARUNG JATI                       2        2   

                                       shelter_code_out  terminal_code_out  \
shelter_name_in shelter_name_out                                             
9E              BERMIS                                1                  1   
                DISPENDA                             11                 11   
                DURI KEPA                             3                  3   
                GN.SAHARI MG.DUA                      2                  2   
                INDOSIAR                             22                 22   
...                                                 ...                ...   
pengajuan tjbrt SAWAH BESAR ARAH KOTA                 1                  1   
                SESKOAL                               2                  2   
                SETIABUDI UTARA AINI                  4                  4   
                VELBAK                                1                  1   
                WARUNG JATI                           2                  2   

                                       waktu_tap_out  Hari  Type_out  \
shelter_name_in shelter_name_out                                       
9E              BERMIS                             1     1         1   
          

In [1313]:
dfm2.head(1)

,idx_in,idx_out,card_number_var,shelter_name_in,shelter_code_in,terminal_code_in,waktu_tap_in,Type_in,shelter_name_out,shelter_code_out,terminal_code_out,waktu_tap_out,Hari,Type_out,coordinates_in,coordinates_out,Kor_in,Kor_out,Ket 2
0,127845,102812,0145200025413503,6H,PPD 464,35304,2023-07-17 20:01:27,TOB,6H,PPD 464,35304,2023-07-17 20:01:31,17,TOB,"-2.54999995231628, -2.54999995231628","-2.54999995231628, -2.54999995231628",NaN,NaN,Complete


In [1314]:
dfm2[dfm2['Ket 2']=='Complete']

,idx_in,idx_out,card_number_var,shelter_name_in,shelter_code_in,terminal_code_in,waktu_tap_in,Type_in,shelter_name_out,shelter_code_out,terminal_code_out,waktu_tap_out,Hari,Type_out,coordinates_in,coordinates_out,Kor_in,Kor_out,Ket 2
0,127845,102812,0145200025413503,6H,PPD 464,35304,2023-07-17 20:01:27,TOB,6H,PPD 464,35304,2023-07-17 20:01:31,17,TOB,"-2.54999995231628, -2.54999995231628","-2.54999995231628, -2.54999995231628",NaN,NaN,Complete
1,16822,9586,0145000144261282,8E,TSW 071,35340,2023-07-17 06:46:53,TOB,8E,TSW 071,35340,2023-07-17 06:46:57,17,TOB,"-6.17056068, -6.17056068","-6.17056068, -6.17056068",NaN,NaN,Complete
2,76801,59489,7546170002051396,5M,TJ 834,25199,2023-07-17 15:10:31,TOB,5M,TJ 834,25199,2023-07-17 15:10:35,17,TOB,"-6.189718, -6.189718","-6.189718, -6.189718",NaN,NaN,Complete
3,27714,17271,0145200008328231,5M,TJ 846,35827,2023-07-17 07:33:55,TOB,5M,TJ 846,35827,2023-07-17 07:33:59,17,TOB,"-6.31278885, -6.31278885","-6.31278885, -6.31278885",NaN,NaN,Complete
4,11627,6157,7546270000034970,8D,KBM220005,35672,2023-07-17 06:23:49,TOB,8D,KBM220005,35672,2023-07-17 06:23:53,17,TOB,"-2.54999995231628, -2.54999995231628","-2.54999995231628, -2.54999995231628",NaN,NaN,Complete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3640034,68877,71245,0145000624522179,12P,MYS 21231,45761,2023-07-24 13:34:16,TOB,12P,MYS 21231,45761,2023-07-24 16:53:51,24,TOB,"-6.13841, -6.13841","-6.15775, -6.15775",NaN,NaN,Complete
3640035,41056,47027,9360885009210145,7B,TJ 288,25137,2023-07-24 08:37:44,TOB,7B,MYS 21275,46322,2023-07-24 11:57:40,24,TOB,"0.383333, 0.383333","-6.24341, -6.24341",NaN,NaN,Complete
3640036,310290,51838,9360885009324219,WALIKOTA JAKARTA TIMUR,11195,10506,2023-07-24 09:56:04,GATE,1A,MYS 21229,45759,2023-07-24 13:16:09,24,TOB,NaN,"-6.12526, -6.12526",KORIDOR 11,NaN,Complete
3640037,47979,49905,0145003300354743,1B,TJ 294,25149,2023-07-24 09:23:18,TOB,9D,MYS 21329,46376,2023-07-24 12:43:33,24,TOB,"0.000000, 0.000000","-6.24552, -6.24552",NaN,NaN,Complete


In [1320]:
df17 = dfm2[dfm2['Hari']==17]
len(df17), len(df17[df17['Ket 2']=='Complete']), round(len(df17[df17['Ket 2']=='Complete'])/len(df17)*100,2)

(542159, 467955, 86.31)

# 6. Guessing the Unmatched: Using Historical Data

# 7. LAIN-LAIN

In [3]:
# df = pd.read_csv('../Juli IT/output4/matched_perday/230803_Matched all_2.csv', index=False)

df = pd.read_csv('../Juli IT/output2/230803_Matched all.csv')

C:\Users\Nadya\AppData\Local\Temp\ipykernel_5352\34009841.py:3: DtypeWarning: Columns (2,3,4,6,7,8,9,11,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Juli IT/output2/230803_Matched all.csv')


MemoryError: Unable to allocate 399. MiB for an array with shape (14, 3736733) and data type object

In [ ]:
df.head(3)

In [ ]:
len(df0[df0['duration']==0]), len(df0)

In [ ]:
df0[df0['card_number_var']==df0[df0['duration']==0]['card_number_var'].iloc[20000]]

In [ ]:
dfa = df0.loc[df0['shelter_code_out'].isna() | df0['shelter_code_out'].isna()]
dfa

# TRYING NEW IDEA (ITERATE (AGAIN) (BUT NEW THIS TIME))

In [44]:
dfin0.head(2)

,Shelter Code,Shelter Name Var,Tap,Terminal Code,Waktu Tap,Card Number Var,Hour,Day,Type,P Latitude Flo,P Longitude Flo,idx_in
347,13241,ADAM MALIK,IN,10592,2023-06-20 00:56:24,6032982512601228,0,20,GATE,NaN,NaN,0
348,13241,ADAM MALIK,IN,10592,2023-06-20 03:36:19,6032986099207697,3,20,GATE,NaN,NaN,1


In [45]:
dfout0.head(2)

,Shelter Code,Shelter Name Var,Tap,Terminal Code,Waktu Tap,Card Number Var,Hour,Day,Type,P Latitude Flo,P Longitude Flo,idx_out
2133570,13241,ADAM MALIK,OUT,10592,2023-06-20 00:04:18,6032984068278724,0,20,GATE,NaN,NaN,0
2133571,13241,ADAM MALIK,OUT,10592,2023-06-20 00:19:17,6013509904417344,0,20,GATE,NaN,NaN,1


In [27]:
# dfout0['']

In [47]:
df0 = pd.concat([dfin0, dfout0])

In [48]:
df0

,Shelter Code,Shelter Name Var,Tap,Terminal Code,Waktu Tap,Card Number Var,Hour,Day,Type,P Latitude Flo,P Longitude Flo,idx_in,idx_out
347,13241,ADAM MALIK,IN,10592,2023-06-20 00:56:24,6032982512601228,0,20,GATE,NaN,NaN,0,NaN
348,13241,ADAM MALIK,IN,10592,2023-06-20 03:36:19,6032986099207697,3,20,GATE,NaN,NaN,1,NaN
349,13241,ADAM MALIK,IN,10592,2023-06-20 04:33:08,145200002311332,4,20,GATE,NaN,NaN,2,NaN
350,13241,ADAM MALIK,IN,10592,2023-06-20 04:39:47,6013500800410855,4,20,GATE,NaN,NaN,3,NaN
351,13241,ADAM MALIK,IN,10592,2023-06-20 04:43:30,1003032206047302,4,20,GATE,NaN,NaN,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378267,B7580TAA,TSW 100,OUT,35399,2023-06-20 21:38:53,9360885003964648,21,20,TOB,-6,107,NaN,424912
1378268,B7580TAA,TSW 100,OUT,35399,2023-06-20 21:38:57,9360885003964540,21,20,TOB,-6,107,NaN,424913
1378269,B7580TAA,TSW 100,OUT,35399,2023-06-20 21:39:37,6032984027966344,21,20,TOB,-6,107,NaN,424914
1378270,B7580TAA,TSW 100,OUT,35399,2023-06-20 21:41:10,9360885008779196,21,20,TOB,-6,107,NaN,424915
